In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_training/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9582 6288 
metadata(1): version
assays(1): ''
rownames(9582): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(6288): V1 V2 ... V6287 V6288
colData names(3): Sample Tissue sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
print(sample)

DataFrame with 6288 rows and 3 columns
           Sample    Tissue  sizeFactor
      <character>  <factor>   <numeric>
V1             V1 Liver        0.344676
V2             V2 Brain       14.798875
V3             V3 Intestine    2.363913
V4             V4 Intestine   12.120910
V5             V5 Liver        0.754843
...           ...       ...         ...
V6284       V6284  Brain    5.56923e-04
V6285       V6285  Lung     1.25359e+01
V6286       V6286  Pancreas 3.50940e-01
V6287       V6287  Adipose  2.45864e+00
V6288       V6288  Sperm    4.92234e+00


In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(vst.count.mtx)

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509603 12.614849 15.519749 14.637853 14.055204 13.449194
ENSMUSG00000000028  8.444428  7.614693 10.420513  9.853543  7.905628  8.286601
ENSMUSG00000000049 20.595033  6.507020  8.077450  7.916962 20.798987  6.163521
ENSMUSG00000000058 10.526181 11.266114  8.766421 11.650309  9.692308 15.196969
ENSMUSG00000000085  9.811443 13.441425 11.548222 12.198297 11.906675 12.810133
ENSMUSG00000000088 16.480555 13.705895 14.766641 14.470710 18.266177 11.941156
ENSMUSG00000000120  9.570225  8.731978  9.042672  9.580352 11.275289  7.829244
ENSMUSG00000000125  4.150751  8.571666 10.636493  4.498008  3.916744  5.114607
ENSMUSG00000000126  7.073363  7.212179  7.601713  7.519445  7.816734 10.184154
ENSMUSG00000000127 11.007871 10.848659 10.921434 10.695664 11.936982 11.026716
ENSMUSG00000000142 10.089229 11.974985 14.516736 11.808314 10.162602 12.502987
ENSMUSG00000000149 14.462026 13.909150 12.471860 12.

ENSMUSG00000000120  8.673436  8.097021  8.534829  7.509745 14.287384  9.950903
ENSMUSG00000000125 10.691888 11.788141  9.107100  8.447599 10.539475  3.795602
ENSMUSG00000000126  7.430537 10.838429  9.501188 11.598598 10.537546  7.411336
ENSMUSG00000000127 10.858316 14.595711 11.648636 10.663087 12.409899 12.113986
ENSMUSG00000000142 11.562486 10.841660 12.306999 10.917297 14.173773 11.560116
ENSMUSG00000000149 14.777297 14.144060 14.153404 14.278217 12.319572 14.565825
ENSMUSG00000000154  5.937776  4.390386  6.871156  6.201857  5.612258 13.509432
ENSMUSG00000000168 14.229457 12.527489 13.048111 13.461766  9.063161 13.037069
ENSMUSG00000000171 13.815139 13.725104 13.049585 13.765870  9.018556 15.051934
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863803 15.551397 15.509006 12.881102 15.083503 12.890134
ENSMUSG00000000028 10.222846  8.291748  9.959167 10.339746  8.465116  8.994184
ENSMUSG00000000049  8.770829 20.313778 19.718144  4.

ENSMUSG00000000168 12.979829 12.311299 13.708315 13.553122 13.405115 14.667265
ENSMUSG00000000171 12.931972 14.804357 16.474337 13.424513 15.704662 15.702937
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582164 12.503303 12.640720 15.584722 12.360589 15.655107
ENSMUSG00000000028  7.948058  9.399669  9.131683  8.711484  9.039425  3.999828
ENSMUSG00000000049  8.154628  8.265493  9.322246 19.485815  7.646570 10.611464
ENSMUSG00000000058 12.562865 13.554198 13.822165 10.170056 13.003293 14.090990
ENSMUSG00000000085 11.816218 13.551972 13.467945 11.023644 12.215782 10.335462
ENSMUSG00000000088 13.873622 18.052361 17.152794 15.238426 16.525558 17.466673
ENSMUSG00000000120  6.316021  7.212504  7.577352 10.595759  8.724745  9.582848
ENSMUSG00000000125  8.244991  3.757661  5.478334  3.866427  5.409441  3.999828
ENSMUSG00000000126 11.836819 12.289934 12.020566  7.005304 11.212689 10.369110
ENSMUSG00000000127 12.502172 11.426191 11.439489 11.

ENSMUSG00000000085 13.377384 11.950847 10.682192  7.102949 11.774735  7.384598
ENSMUSG00000000088 13.748764 17.098015 14.128457 13.834486 17.289888 12.944806
ENSMUSG00000000120  8.850066  8.496348  8.831413  7.477018  9.084898  7.384598
ENSMUSG00000000125  3.836952  3.591454  8.891628 12.960936  3.682280  7.384598
ENSMUSG00000000126  8.656505 10.409246  7.762958  7.648244  8.997958  7.384598
ENSMUSG00000000127 13.637552 11.891918 11.251967 12.333786 10.935605 13.652287
ENSMUSG00000000142 10.697952 10.144489 13.998690 12.951234 11.356774 19.361243
ENSMUSG00000000149 12.747787 15.369466 11.971141 13.568618 14.846623 15.746155
ENSMUSG00000000154  8.593219  7.100625 12.085732  9.844779  8.249874  7.384598
ENSMUSG00000000168 15.018110 16.758113 12.602273 12.613954 17.571565  7.384598
ENSMUSG00000000171 13.386707 16.754295 14.942899 13.933354 17.264503 17.441889
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743406 15.700681 12.025687 15.1

ENSMUSG00000000154 16.730380 10.771127  6.903922 11.683438  5.189053  6.299277
ENSMUSG00000000168 13.494395 14.510930 14.029956 13.278151  3.657881 13.508191
ENSMUSG00000000171 15.765563 12.158348 13.468094 13.747092  9.439631 13.214819
                        V157      V158      V159      V160      V161      V162
ENSMUSG00000000001 13.354353 13.238306 15.007390 11.390855 14.207677 14.211857
ENSMUSG00000000028  9.545085  9.728822  8.844120  9.679816  9.110549  9.520635
ENSMUSG00000000049  6.079527  5.235133  6.067973  7.860408  7.890098  6.740676
ENSMUSG00000000058 11.852683 11.818358 17.602474  8.845266 14.881037 17.624887
ENSMUSG00000000085 11.783431 11.481409 11.641594 14.091966 13.336970 11.509742
ENSMUSG00000000088 13.884200 14.158386 12.969517 12.757092 19.039605 13.464052
ENSMUSG00000000120  8.740923  8.284987  7.030068 10.070263  7.559991  7.797192
ENSMUSG00000000125  5.990738  6.291702  5.413787 10.983678  3.887102

...

 18.697137 13.614313 13.958523 10.729975
ENSMUSG00000000171 12.758753 16.983335 18.581597 14.670834 13.016333 10.451634
                       V5647     V5648     V5649     V5650     V5651    V5652
ENSMUSG00000000001 12.948829 15.080402 14.758726 15.514706 12.500639 10.59471
ENSMUSG00000000028 10.271370  8.949848  8.235356  8.941905  9.186402 10.59471
ENSMUSG00000000049  7.079207 19.958483  9.590449 20.710171  7.669102 10.59471
ENSMUSG00000000058 11.138237 10.753054 12.235074 10.439658 12.526324 10.59471
ENSMUSG00000000085  9.952714 10.693028 11.474245 10.974635 11.667201 10.59471
ENSMUSG00000000088 16.365801 15.341348 15.410208 15.494370 13.120826 10.59471
ENSMUSG00000000120  8.965809  9.839791  4.809897  9.488621 11.759337 10.59471
ENSMUSG00000000125  5.393788  6.161032  4.809897  3.782271  6.186044 10.59471
ENSMUSG00000000126 10.914105  7.853481  8.256013  6.712826  7.841123 10.59471
ENSMUSG00000000127 11.950809 11.284186 11.021420 10.885001 12.992324 10.59471
ENSMUSG00000000142  9.

ENSMUSG00000000088 14.433006 15.600572 15.561742 16.141178 15.943114 17.483566
ENSMUSG00000000120  9.027438 11.176839  9.407067 13.468524  8.200300  8.108841
ENSMUSG00000000125  9.630778  3.725192  3.693130  3.556968  4.047108  3.772803
ENSMUSG00000000126 10.209682  7.146574  8.885203  7.212471  8.842188 12.098179
ENSMUSG00000000127 11.755499 10.869473 11.431928  9.788794 10.997882 11.250438
ENSMUSG00000000142 11.759926 10.908074 10.678008  9.449106 11.084513 11.487493
ENSMUSG00000000149 13.270983 15.525954 16.420190 14.043502 13.635639 12.925416
ENSMUSG00000000154  6.271146 17.196761 14.222987 17.023471  9.906803  7.504950
ENSMUSG00000000168 13.574403 14.523171 12.903046 12.984605 16.684495 16.497876
ENSMUSG00000000171 13.108977 16.207983 16.138571 16.549009 15.851892 16.869459
                       V5689     V5690     V5691     V5692     V5693     V5694
ENSMUSG00000000001 15.070933 14.062434 15.360498 15.187182 14.217014 12.628034
ENSMUSG00000000028  8.101083  8.208157  8.937950  9.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 14.269760
ENSMUSG00000000154  8.312094  9.397437  7.407997 16.229532  5.126582 15.874942
ENSMUSG00000000168 15.680692 15.286353 16.933708 14.964366 13.690419 14.565806
ENSMUSG00000000171 14.744223 14.689071 17.924619 15.720909 13.376009 15.865502
                       V5845     V5846     V5847     V5848     V5849     V5850
ENSMUSG00000000001 15.389398 12.140992 14.585045 15.177566 14.814331 11.935505
ENSMUSG00000000028  8.036832  9.735816  9.370476  7.217486  8.159790  8.578878
ENSMUSG00000000049 19.957547  6.290976 19.815114 20.243798 20.081354  7.179382
ENSMUSG00000000058 10.415584 12.921270 10.324815 11.665043  8.696515 11.352828
ENSMUSG00000000085 10.053590 13.771550 11.715414 10.621208 11.556443 12.667172
ENSMUSG00000000088 15.315284 18.589252 14.312266 13.952978 15.885033 12.033371
ENSMUSG00000000120  8.651276  7.576862 11.880299 10.203175 10.918045  9.633705
ENSMUSG00000000125  3.914931  3.557840  3.788450  4.679610  3.626776  9.946143
ENSMUSG00000000126  8.806099 10.644717  7

ENSMUSG00000000058 14.720182 12.110441  4.124149 11.622081 12.352810  3.978680
ENSMUSG00000000085 13.364272 13.101590  4.124149 13.161203 11.606663 13.717394
ENSMUSG00000000088 18.751527 13.750266 13.641725 13.791994 13.533082 13.312910
ENSMUSG00000000120  7.405602  9.273186 10.695175 10.100258  8.970483  3.978680
ENSMUSG00000000125  5.299534  9.422259 14.934174  8.295385  9.067309  3.978680
ENSMUSG00000000126 11.464689 10.877215 10.628276  6.917790 10.696312  3.978680
ENSMUSG00000000127 12.795606 10.044216  4.124149 11.147808 12.529519  8.067998
ENSMUSG00000000142  9.416689 11.803463 16.020927 11.465660 11.847470 14.121120
ENSMUSG00000000149 11.618135 14.176370  4.124149 13.993243 13.563878 12.925773
ENSMUSG00000000154  6.719971  6.842929  8.848379  6.446902  6.510754  3.978680
ENSMUSG00000000168 17.426836 13.637039 13.024993 13.836789 14.238963 15.822619
ENSMUSG00000000171 17.360086 13.351334 14.478706 13.631175 13.343317 15.103720
                       V5887     V5888     V5889    

In [7]:
%%R
GEO_model_training_tissue_detail.vec <-
   readRDS(paste(OUT_DIR, "GEO_model_training_tissue_detail_vec.Rds", sep = ""))
print(GEO_model_training_tissue_detail.vec)

In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls.Rds", sep = ""))
print(rxn2ensembls.nls)

 "ENSMUSG00000000617" "ENSMUSG00000000766" "ENSMUSG00000003974"
 [4] "ENSMUSG00000004207" "ENSMUSG00000004366" "ENSMUSG00000004630"
 [7] "ENSMUSG00000018920" "ENSMUSG00000018927" "ENSMUSG00000018930"
[10] "ENSMUSG00000019122" "ENSMUSG00000020660" "ENSMUSG00000020676"
[13] "ENSMUSG00000020793" "ENSMUSG00000021070" "ENSMUSG00000021303"
[16] "ENSMUSG00000024462" "ENSMUSG00000024553" "ENSMUSG00000024907"
[19] "ENSMUSG00000025804" "ENSMUSG00000026237" "ENSMUSG00000027883"
[22] "ENSMUSG00000028971" "ENSMUSG00000029064" "ENSMUSG00000029236"
[25] "ENSMUSG00000029417" "ENSMUSG00000029530" "ENSMUSG00000029663"
[28] "ENSMUSG00000029713" "ENSMUSG00000029819" "ENSMUSG00000031778"
[31] "ENSMUSG00000031861" "ENSMUSG00000031980" "ENSMUSG00000033774"
[34] "ENSMUSG00000034117" "ENSMUSG00000034786" "ENSMUSG00000034855"
[37] "ENSMUSG00000035042" "ENSMUSG00000036353" "ENSMUSG00000036362"
[40] "ENSMUSG00000036381" "ENSMUSG00000036402" "ENSMUSG00000036437"
[43] "ENSMUSG00000037346" "ENSMUSG00000039809" "ENSM

[1] "ENSMUSG00000000708" "ENSMUSG00000020918" "ENSMUSG00000061143"

$`R-MMU-2247925`
[1] "ENSMUSG00000000708"

$`R-MMU-4396387`
[1] "ENSMUSG00000000708" "ENSMUSG00000019789"

$`R-MMU-5211235`
[1] "ENSMUSG00000000708" "ENSMUSG00000031832" "ENSMUSG00000031939"

$`R-MMU-5250930`
 [1] "ENSMUSG00000000708" "ENSMUSG00000002748" "ENSMUSG00000004667"
 [4] "ENSMUSG00000020561" "ENSMUSG00000020918" "ENSMUSG00000021377"
 [7] "ENSMUSG00000028318" "ENSMUSG00000031715" "ENSMUSG00000031832"
[10] "ENSMUSG00000031939" "ENSMUSG00000036315" "ENSMUSG00000038489"
[13] "ENSMUSG00000040463" "ENSMUSG00000045996"

$`R-MMU-5691381`
 [1] "ENSMUSG00000000708" "ENSMUSG00000008348" "ENSMUSG00000019505"
 [4] "ENSMUSG00000020460" "ENSMUSG00000061615" "ENSMUSG00000061991"
 [7] "ENSMUSG00000062627" "ENSMUSG00000063021" "ENSMUSG00000063689"
[10] "ENSMUSG00000068240" "ENSMUSG00000068855" "ENSMUSG00000069270"
[13] "ENSMUSG00000069272" "ENSMUSG00000069301" "ENSMUSG00000069302"
[16] "ENSMUSG00000069309" "ENSMUSG00000078851"

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[16] "ENSMUSG00000028495" "ENSMUSG00000029614" "ENSMUSG00000030655"
[19] "ENSMUSG00000030744" "ENSMUSG00000032399" "ENSMUSG00000034681"
[22] "ENSMUSG00000034892" "ENSMUSG00000038374" "ENSMUSG00000038398"
[25] "ENSMUSG00000039209" "ENSMUSG00000039221" "ENSMUSG00000040952"
[28] "ENSMUSG00000041453" "ENSMUSG00000047215" "ENSMUSG00000052146"
[31] "ENSMUSG00000057841" "ENSMUSG00000057863" "ENSMUSG00000058301"
[34] "ENSMUSG00000060636" "ENSMUSG00000060938" "ENSMUSG00000061787"
[37] "ENSMUSG00000061983" "ENSMUSG00000062006" "ENSMUSG00000062203"
[40] "ENSMUSG00000062328" "ENSMUSG00000062647" "ENSMUSG00000062997"
[43] "ENSMUSG00000063316" "ENSMUSG00000068240" "ENSMUSG00000079435"
[46] "ENSMUSG00000090137" "ENSMUSG00000090733" "ENSMUSG00000090862"
[49] "ENSMUSG00000098274"

$`R-MMU-927832`
 [1] "ENSMUSG00000000740" "ENSMUSG00000002210" "ENSMUSG00000002500"
 [4] "ENSMUSG00000003970" "ENSMUSG00000007892" "ENSMUSG00000009927"
 [7] "ENSMUSG00000020460" "ENSMUSG00000020495" "ENSMUSG00000022283"
[10] 

 [4] "ENSMUSG00000024854" "ENSMUSG00000027342" "ENSMUSG00000029191"
 [7] "ENSMUSG00000029363" "ENSMUSG00000030042" "ENSMUSG00000030726"
[10] "ENSMUSG00000031536" "ENSMUSG00000033970"

$`R-MMU-110368`
 [1] "ENSMUSG00000000751" "ENSMUSG00000020471" "ENSMUSG00000020974"
 [4] "ENSMUSG00000024854" "ENSMUSG00000027342" "ENSMUSG00000029191"
 [7] "ENSMUSG00000029363" "ENSMUSG00000030042" "ENSMUSG00000030726"
[10] "ENSMUSG00000033970"

$`R-MMU-110371`
 [1] "ENSMUSG00000000751" "ENSMUSG00000020471" "ENSMUSG00000020974"
 [4] "ENSMUSG00000024854" "ENSMUSG00000027342" "ENSMUSG00000029191"
 [7] "ENSMUSG00000029363" "ENSMUSG00000030042" "ENSMUSG00000030726"
[10] "ENSMUSG00000033970"

$`R-MMU-174441`
[1] "ENSMUSG00000000751" "ENSMUSG00000020471" "ENSMUSG00000024854"
[4] "ENSMUSG00000027342" "ENSMUSG00000029676" "ENSMUSG00000030726"
[7] "ENSMUSG00000033430"

$`R-MMU-174445`
[1] "ENSMUSG00000000751" "ENSMUSG00000020471" "ENSMUSG00000024854"
[4] "ENSMUSG00000027342" "ENSMUSG00000029676" "ENSMUSG000000307

...

[1] "ENSMUSG00000038393"

$`R-MMU-1676005`
[1] "ENSMUSG00000038417"

$`R-MMU-1676020`
[1] "ENSMUSG00000038417"

$`R-MMU-1676174`
[1] "ENSMUSG00000038417"

$`R-MMU-5690152`
[1] "ENSMUSG00000038429"

$`R-MMU-9754974`
[1] "ENSMUSG00000039197"

$`R-MMU-8848370`
[1] "ENSMUSG00000039217"

$`R-MMU-8848392`
[1] "ENSMUSG00000039217"

$`R-MMU-8848395`
[1] "ENSMUSG00000039217"

$`R-MMU-8931648`
[1] "ENSMUSG00000039242" "ENSMUSG00000039952"

$`R-MMU-5682769`
[1] "ENSMUSG00000039252" "ENSMUSG00000040537"

$`R-MMU-6788855`
[1] "ENSMUSG00000039253"

$`R-MMU-5615637`
[1] "ENSMUSG00000039254" "ENSMUSG00000039952"

$`R-MMU-141139`
[1] "ENSMUSG00000039304"

$`R-MMU-75238`
[1] "ENSMUSG00000039304"

$`R-MMU-8850040`
[1] "ENSMUSG00000039318"

$`R-MMU-8850041`
[1] "ENSMUSG00000039318"

$`R-MMU-381412`
[1] "ENSMUSG00000039323"

$`R-MMU-8932275`
[1] "ENSMUSG00000039345"

$`R-MMU-9603984`
[1] "ENSMUSG00000039357"

$`R-MMU-5675376`
[1] "ENSMUSG00000039384" "ENSMUSG00000063065"

$`R-MMU-9629360`


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





$`R-MMU-170825`
[1] "ENSMUSG00000041798"

$`R-MMU-8870352`
[1] "ENSMUSG00000043885"

$`R-MMU-8870354`
[1] "ENSMUSG00000043885"

$`R-MMU-419428`
[1] "ENSMUSG00000043895" "ENSMUSG00000044199"

$`R-MMU-2997723`
[1] "ENSMUSG00000043909"

$`R-MMU-3296301`
[1] "ENSMUSG00000043953" "ENSMUSG00000071005"

$`R-MMU-9830451`
[1] "ENSMUSG00000043969"

$`R-MMU-975829`
[1] "ENSMUSG00000043998"

$`R-MMU-70609`
[1] "ENSMUSG00000044005"

$`R-MMU-373086`
[1] "ENSMUSG00000044052" "ENSMUSG00000074715" "ENSMUSG00000096826"

$`R-MMU-6805943`
[1] "ENSMUSG00000044080"

$`R-MMU-8848316`
[1] "ENSMUSG00000044103" "ENSMUSG00000070942"

$`R-MMU-211178`
[1] "ENSMUSG00000044167"

$`R-MMU-9614414`
[1] "ENSMUSG00000044167"

$`R-MMU-9621286`
[1] "ENSMUSG00000044167"

$`R-MMU-9623027`
[1] "ENSMUSG00000044167"

$`R-MMU-9623032`
[1] "ENSMUSG00000044167"

$`R-MMU-9625830`
[1] "ENSMUSG00000044167"

$`R-MMU-9625847`
[1] "ENSMUSG00000044167"

$`R-MMU-9626928`
[1] "ENSMUSG00000044167"

$`R-MMU-9626966`
[1] "ENSMUSG00000044167

[1] "ENSMUSG00000052738" "ENSMUSG00000061838"

$`R-MMU-2744361`
[1] "ENSMUSG00000052819"

$`R-MMU-2752067`
[1] "ENSMUSG00000052819"

$`R-MMU-9684627`
[1] "ENSMUSG00000052821"

$`R-MMU-9690439`
[1] "ENSMUSG00000052821"

$`R-MMU-2990833`
[1] "ENSMUSG00000052833" "ENSMUSG00000052997"

$`R-MMU-2993780`
[1] "ENSMUSG00000052833" "ENSMUSG00000052997"

$`R-MMU-114553`
[1] "ENSMUSG00000052889" "ENSMUSG00000078816"

$`R-MMU-1168373`
[1] "ENSMUSG00000052889"

$`R-MMU-5223304`
[1] "ENSMUSG00000052889"

$`R-MMU-211966`
[1] "ENSMUSG00000052914" "ENSMUSG00000061740"

$`R-MMU-76456`
[1] "ENSMUSG00000052914" "ENSMUSG00000061740"

$`R-MMU-4551446`
[1] "ENSMUSG00000052920"

$`R-MMU-6807585`
[1] "ENSMUSG00000052922"

$`R-MMU-76453`
[1] "ENSMUSG00000052974" "ENSMUSG00000074254" "ENSMUSG00000091867"

$`R-MMU-390912`
[1] "ENSMUSG00000053004"

$`R-MMU-9692539`
[1] "ENSMUSG00000053004"

$`R-MMU-202709`
[1] "ENSMUSG00000053062"

$`R-MMU-1254248`
[1] "ENSMUSG00000053110" "ENSMUSG00000062209"

$`R-MMU-1254251`
[1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(rxns)

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(GEO_model_training_tissue_detail.vec))
print(toi_indices)


[3921] 3921 3922 3923 3924 3925 3926 3927 3928 3929 3930 3931 3932 3933 3934
[3935] 3935 3936 3937 3938 3939 3940 3941 3942 3943 3944 3945 3946 3947 3948
[3949] 3949 3950 3951 3952 3953 3954 3955 3956 3957 3958 3959 3960 3961 3962
[3963] 3963 3964 3965 3966 3967 3968 3969 3970 3971 3972 3973 3974 3975 3976
[3977] 3977 3978 3979 3980 3981 3982 3983 3984 3985 3986 3987 3988 3989 3990
[3991] 3991 3992 3993 3994 3995 3996 3997 3998 3999 4000 4001 4002 4003 4004
[4005] 4005 4006 4007 4008 4009 4010 4011 4012 4013 4014 4015 4016 4017 4018
[4019] 4019 4020 4021 4022 4023 4024 4025 4026 4027 4028 4029 4030 4031 4032
[4033] 4033 4034 4035 4036 4037 4038 4039 4040 4041 4042 4043 4044 4045 4046
[4047] 4047 4048 4049 4050 4051 4052 4053 4054 4055 4056 4057 4058 4059 4060
[4061] 4061 4062 4063 4064 4065 4066 4067 4068 4069 4070 4071 4072 4073 4074
[4075] 4075 4076 4077 4078 4079 4080 4081 4082 4083 4084 4085 4086 4087 4088
[4089] 4089 4090 4091 4092 4093 4094 4095 4096 4097 4098 4099 4100 4101 410

[5405] 5405 5406 5407 5408 5409 5410 5411 5412 5413 5414 5415 5416 5417 5418
[5419] 5419 5420 5421 5422 5423 5424 5425 5426 5427 5428 5429 5430 5431 5432
[5433] 5433 5434 5435 5436 5437 5438 5439 5440 5441 5442 5443 5444 5445 5446
[5447] 5447 5448 5449 5450 5451 5452 5453 5454 5455 5456 5457 5458 5459 5460
[5461] 5461 5462 5463 5464 5465 5466 5467 5468 5469 5470 5471 5472 5473 5474
[5475] 5475 5476 5477 5478 5479 5480 5481 5482 5483 5484 5485 5486 5487 5488
[5489] 5489 5490 5491 5492 5493 5494 5495 5496 5497 5498 5499 5500 5501 5502
[5503] 5503 5504 5505 5506 5507 5508 5509 5510 5511 5512 5513 5514 5515 5516
[5517] 5517 5518 5519 5520

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
%%R
GEO_model_training_tissue_detail_vec_tis_of_interest <-
   GEO_model_training_tissue_detail.vec[toi_indices]
print(GEO_model_training_tissue_detail_vec_tis_of_interest)

   [1] "Liver"         "Brain"         "Intestine"     "Intestine"    
   [5] "Liver"         "Lung"          "Intestine"     "Liver"        
   [9] "Brain"         "Muscle"        "Liver"         "Liver"        
  [13] "Liver"         "Liver"         "Liver"         "Liver"        
  [17] "Brain"         "Muscle"        "Liver"         "Liver"        
  [21] "Testes"        "Heart"         "Skin"          "Brain"        
  [25] "Liver"         "Liver"         "Kidney"        "Testes"       
  [29] "Muscle"        "Skin"          "Intestine"     "Uterus"       
  [33] "Uterus"        "Liver"         "Muscle"        "Liver"        
  [37] "Brain"         "Sperm"         "Testes"        "Brain"        
  [41] "Eye"           "Liver"         "Intestine"     "Liver"        
  [45] "Liver"         "Adipose"       "Liver"         "Muscle"       
  [49] "Heart"         "Adipose"       "Lung"          "Eye"          
  [53] "Liver"         "Liver"         "Adipose"       "Adrenal"      
  [57]

 [461] "Brain"         "Ovary"         "Liver"         "Kidney"       
 [465] "Pancreas"      "Adipose"       "Intestine"     "Kidney"       
 [469] "Liver"         "Brain"         "Lung"          "Liver"        
 [473] "Brain"         "Intestine"     "Intestine"     "Liver"        
 [477] "Kidney"        "Adipose"       "Liver"         "Liver"        
 [481] "Liver"         "Muscle"        "Liver"         "Liver"        
 [485] "Liver"         "Muscle"        "Adipose"       "Adipose"      
 [489] "Adipose"       "MOE"           "Liver"         "Liver"        
 [493] "Adipose"       "Olfactory"     "Testes"        "Lung"         
 [497] "Testes"        "Liver"         "Kidney"        "Brain"        
 [501] "Liver"         "Uterus"        "Brain"         "Brain"        
 [505] "Liver"         "Kidney"        "Intestine"     "Liver"        
 [509] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [513] "Liver"         "Brain"         "Liver"         "Adipose"      
 [517]

 [921] "Brain"         "Muscle"        "Kidney"        "Brain"        
 [925] "Brain"         "Testes"        "Brain"         "Heart"        
 [929] "Liver"         "Adipose"       "Liver"         "Intestine"    
 [933] "Brain"         "Adipose"       "Liver"         "Brain"        
 [937] "Brain"         "Muscle"        "Liver"         "Muscle"       
 [941] "Liver"         "Bone"          "Heart"         "Brain"        
 [945] "Testes"        "Muscle"        "Eye"           "Liver"        
 [949] "Sperm"         "Liver"         "Uterus"        "Kidney"       
 [953] "Adipose"       "Intestine"     "Skin"          "Brain"        
 [957] "Kidney"        "Liver"         "Brain"         "Intestine"    
 [961] "Liver"         "Brain"         "VNO"           "Olfactory"    
 [965] "Liver"         "Liver"         "Brain"         "Sperm"        
 [969] "Brain"         "Liver"         "Liver"         "Liver"        
 [973] "Liver"         "Intestine"     "Intestine"     "Brain"        
 [977]

[1381] "Brain"         "Liver"         "Liver"         "Liver"        
[1385] "Skin"          "Testes"        "Liver"         "Brain"        
[1389] "Brain"         "Adipose"       "Brain"         "Muscle"       
[1393] "Adipose"       "Testes"        "Heart"         "Intestine"    
[1397] "Intestine"     "Brain"         "Skin"          "Muscle"       
[1401] "Muscle"        "Brain"         "Liver"         "Lung"         
[1405] "Liver"         "Brain"         "Muscle"        "Bone"         
[1409] "Liver"         "Brain"         "Sperm"         "Lung"         
[1413] "Liver"         "MOE"           "Liver"         "Brain"        
[1417] "Brain"         "Brain"         "Liver"         "Liver"        
[1421] "Adipose"       "Adipose"       "Liver"         "Bone"         
[1425] "Bone"          "Liver"         "Brain"         "Brain"        
[1429] "Liver"         "Pancreas"      "Liver"         "Pancreas"     
[1433] "Pancreas"      "Liver"         "Pancreas"      "Adipose"      
[1437]

[1841] "Brain"         "Adipose"       "Sperm"         "Heart"        
[1845] "Ovary"         "Nerve"         "Heart"         "Brain"        
[1849] "Adipose"       "Brain"         "Intestine"     "Testes"       
[1853] "Eye"           "Liver"         "Nerve"         "Brain"        
[1857] "Liver"         "Muscle"        "Spinal cord"   "Lung"         
[1861] "Liver"         "Liver"         "Kidney"        "Liver"        
[1865] "Intestine"     "Adipose"       "Kidney"        "Brain"        
[1869] "Adipose"       "Intestine"     "Pancreas"      "Testes"       
[1873] "Liver"         "Ovary"         "Liver"         "Adipose"      
[1877] "Liver"         "Liver"         "MOE"           "Brain"        
[1881] "Brain"         "Kidney"        "Brain"         "Nerve"        
[1885] "Brain"         "Liver"         "Brain"         "Kidney"       
[1889] "Testes"        "Intestine"     "Liver"         "Stomach"      
[1893] "Intestine"     "Ovary"         "Adipose"       "Pancreas"     
[1897]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "Liver"         "Adipose"      
[4513] "Lung"          "Stomach"       "Testes"        "Heart"        
[4517] "Brain"         "Skin"          "Testes"        "Brain"        
[4521] "Brain"         "Liver"         "Brain"         "Brain"        
[4525] "Kidney"        "Liver"         "Liver"         "Intestine"    
[4529] "Tendon"        "Aorta"         "Muscle"        "Kidney"       
[4533] "Brain"         "Brain"         "Liver"         "Brain"        
[4537] "Kidney"        "Testes"        "Liver"         "Liver"        
[4541] "Brain"         "Muscle"        "Intestine"     "Eye"          
[4545] "Liver"         "Liver"         "Brain"         "Brain"        
[4549] "Brain"         "Liver"         "Liver"         "Intestine"    
[4553] "Brain"         "VNO"           "Lung"          "Brain"        
[4557] "Intestine"     "Brain"         "Pancreas"      "Intestine"    
[4561] "Intestine"     "Eye"           "Adipose"       "Brain"        
[4565] "Liver"         "Liver"         "Inte

[4969] "Adipose"       "MOE"           "Uterus"        "Testes"       
[4973] "Adipose"       "Kidney"        "Liver"         "Testes"       
[4977] "Intestine"     "Adipose"       "Heart"         "Intestine"    
[4981] "Brain"         "Brain"         "Liver"         "Intestine"    
[4985] "Liver"         "Liver"         "Muscle"        "Liver"        
[4989] "Spinal cord"   "Skin"          "Brain"         "Liver"        
[4993] "Mammary gland" "Liver"         "Brain"         "Liver"        
[4997] "Brain"         "Liver"         "Liver"         "Liver"        
[5001] "Liver"         "Liver"         "Sperm"         "Mammary gland"
[5005] "Brain"         "Skin"          "Brain"         "Kidney"       
[5009] "Pancreas"      "Brain"         "Kidney"        "Muscle"       
[5013] "Liver"         "Liver"         "Eye"           "Brain"        
[5017] "Liver"         "Liver"         "Testes"        "Liver"        
[5021] "Brain"         "Intestine"     "Lung"          "Liver"        
[5025]

[5429] "Liver"         "Adipose"       "Liver"         "Liver"        
[5433] "Liver"         "Adipose"       "Liver"         "Liver"        
[5437] "Intestine"     "Liver"         "Liver"         "Pancreas"     
[5441] "Brain"         "Ovary"         "Ovary"         "Liver"        
[5445] "Liver"         "Brain"         "Skin"          "Lung"         
[5449] "Intestine"     "Aorta"         "Pancreas"      "Brain"        
[5453] "Pancreas"      "Liver"         "Liver"         "Brain"        
[5457] "Brain"         "Brain"         "Liver"         "Liver"        
[5461] "Spinal cord"   "Liver"         "Heart"         "Intestine"    
[5465] "Intestine"     "Eye"           "Adipose"       "Adipose"      
[5469] "Liver"         "Liver"         "Liver"         "Liver"        
[5473] "Brain"         "Sperm"         "Liver"         "Brain"        
[5477] "Brain"         "Liver"         "Adipose"       "Liver"        
[5481] "Intestine"     "Liver"         "Kidney"        "Brain"        
[5485]

[5889] "Brain"         "Adipose"       "Adipose"       "Kidney"       
[5893] "VNO"           "Pancreas"      "Liver"         "Aorta"        
[5897] "Liver"         "Brain"         "Brain"         "Brain"        
[5901] "Adipose"       "Muscle"        "Kidney"        "Ovary"        
[5905] "Tendon"        "Liver"         "Brain"         "Liver"        
[5909] "Liver"         "Brain"         "Kidney"        "Brain"        
[5913] "Liver"         "Brain"         "VNO"           "Muscle"       
[5917] "Brain"         "Pancreas"      "Adipose"       "Brain"        
[5921] "Spinal cord"   "Tendon"        "Brain"         "Liver"        
[5925] "Liver"         "Eye"           "Muscle"        "Adipose"      
[5929] "Liver"         "Liver"         "Adipose"       "Liver"        
[5933] "MOE"           "Aorta"         "Liver"         "Kidney"       
[5937] "Adipose"       "Liver"         "Liver"         "VNO"          
[5941] "Nerve"         "MOE"           "Liver"         "Brain"        
[5945]

In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509603 12.614849 15.519749 14.637853 14.055204 13.449194
ENSMUSG00000000028  8.444428  7.614693 10.420513  9.853543  7.905628  8.286601
ENSMUSG00000000049 20.595033  6.507020  8.077450  7.916962 20.798987  6.163521
ENSMUSG00000000058 10.526181 11.266114  8.766421 11.650309  9.692308 15.196969
ENSMUSG00000000085  9.811443 13.441425 11.548222 12.198297 11.906675 12.810133
ENSMUSG00000000088 16.480555 13.705895 14.766641 14.470710 18.266177 11.941156
ENSMUSG00000000120  9.570225  8.731978  9.042672  9.580352 11.275289  7.829244
ENSMUSG00000000125  4.150751  8.571666 10.636493  4.498008  3.916744  5.114607
ENSMUSG00000000126  7.073363  7.212179  7.601713  7.519445  7.816734 10.184154
ENSMUSG00000000127 11.007871 10.848659 10.921434 10.695664 11.936982 11.026716
ENSMUSG00000000142 10.089229 11.974985 14.516736 11.808314 10.162602 12.502987
ENSMUSG00000000149 14.462026 13.909150 12.471860 12.

ENSMUSG00000000120  8.673436  8.097021  8.534829  7.509745 14.287384  9.950903
ENSMUSG00000000125 10.691888 11.788141  9.107100  8.447599 10.539475  3.795602
ENSMUSG00000000126  7.430537 10.838429  9.501188 11.598598 10.537546  7.411336
ENSMUSG00000000127 10.858316 14.595711 11.648636 10.663087 12.409899 12.113986
ENSMUSG00000000142 11.562486 10.841660 12.306999 10.917297 14.173773 11.560116
ENSMUSG00000000149 14.777297 14.144060 14.153404 14.278217 12.319572 14.565825
ENSMUSG00000000154  5.937776  4.390386  6.871156  6.201857  5.612258 13.509432
ENSMUSG00000000168 14.229457 12.527489 13.048111 13.461766  9.063161 13.037069
ENSMUSG00000000171 13.815139 13.725104 13.049585 13.765870  9.018556 15.051934
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863803 15.551397 15.509006 12.881102 15.083503 12.890134
ENSMUSG00000000028 10.222846  8.291748  9.959167 10.339746  8.465116  8.994184
ENSMUSG00000000049  8.770829 20.313778 19.718144  4.

ENSMUSG00000000168 12.979829 12.311299 13.708315 13.553122 13.405115 14.667265
ENSMUSG00000000171 12.931972 14.804357 16.474337 13.424513 15.704662 15.702937
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582164 12.503303 12.640720 15.584722 12.360589 15.655107
ENSMUSG00000000028  7.948058  9.399669  9.131683  8.711484  9.039425  3.999828
ENSMUSG00000000049  8.154628  8.265493  9.322246 19.485815  7.646570 10.611464
ENSMUSG00000000058 12.562865 13.554198 13.822165 10.170056 13.003293 14.090990
ENSMUSG00000000085 11.816218 13.551972 13.467945 11.023644 12.215782 10.335462
ENSMUSG00000000088 13.873622 18.052361 17.152794 15.238426 16.525558 17.466673
ENSMUSG00000000120  6.316021  7.212504  7.577352 10.595759  8.724745  9.582848
ENSMUSG00000000125  8.244991  3.757661  5.478334  3.866427  5.409441  3.999828
ENSMUSG00000000126 11.836819 12.289934 12.020566  7.005304 11.212689 10.369110
ENSMUSG00000000127 12.502172 11.426191 11.439489 11.

ENSMUSG00000000085 13.377384 11.950847 10.682192  7.102949 11.774735  7.384598
ENSMUSG00000000088 13.748764 17.098015 14.128457 13.834486 17.289888 12.944806
ENSMUSG00000000120  8.850066  8.496348  8.831413  7.477018  9.084898  7.384598
ENSMUSG00000000125  3.836952  3.591454  8.891628 12.960936  3.682280  7.384598
ENSMUSG00000000126  8.656505 10.409246  7.762958  7.648244  8.997958  7.384598
ENSMUSG00000000127 13.637552 11.891918 11.251967 12.333786 10.935605 13.652287
ENSMUSG00000000142 10.697952 10.144489 13.998690 12.951234 11.356774 19.361243
ENSMUSG00000000149 12.747787 15.369466 11.971141 13.568618 14.846623 15.746155
ENSMUSG00000000154  8.593219  7.100625 12.085732  9.844779  8.249874  7.384598
ENSMUSG00000000168 15.018110 16.758113 12.602273 12.613954 17.571565  7.384598
ENSMUSG00000000171 13.386707 16.754295 14.942899 13.933354 17.264503 17.441889
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743406 15.700681 12.025687 15.1

ENSMUSG00000000154 16.730380 10.771127  6.903922 11.683438  5.189053  6.299277
ENSMUSG00000000168 13.494395 14.510930 14.029956 13.278151  3.657881 13.508191
ENSMUSG00000000171 15.765563 12.158348 13.468094 13.747092  9.439631 13.214819
                        V157      V158      V159      V160      V161      V162
ENSMUSG00000000001 13.354353 13.238306 15.007390 11.390855 14.207677 14.211857
ENSMUSG00000000028  9.545085  9.728822  8.844120  9.679816  9.110549  9.520635
ENSMUSG00000000049  6.079527  5.235133  6.067973  7.860408  7.890098  6.740676
ENSMUSG00000000058 11.852683 11.818358 17.602474  8.845266 14.881037 17.624887
ENSMUSG00000000085 11.783431 11.481409 11.641594 14.091966 13.336970 11.509742
ENSMUSG00000000088 13.884200 14.158386 12.969517 12.757092 19.039605 13.464052
ENSMUSG00000000120  8.740923  8.284987  7.030068 10.070263  7.559991  7.797192
ENSMUSG00000000125  5.990738  6.291702  5.413787 10.983678  3.887102

...

ENSMUSG00000000171 12.910409 10.53489 16.412989 15.009765 16.040439 14.490071
                       V5755     V5756     V5757     V5758     V5759     V5760
ENSMUSG00000000001 13.398911 15.204575 17.273045 14.732695 15.053841 13.467482
ENSMUSG00000000028  8.996418  7.286938 10.041025  9.915574  8.350012  9.547311
ENSMUSG00000000049  9.260419 19.719244 21.585620  7.540312 20.741759  4.789428
ENSMUSG00000000058 14.523648 10.023504  3.929527  3.881948  9.652602 11.270817
ENSMUSG00000000085 12.869122  9.757224 11.570142  3.881948 11.067579 11.805125
ENSMUSG00000000088 12.232685 15.333613  7.938911 15.927804 16.111395 13.956466
ENSMUSG00000000120 14.523811  9.557207 10.282600  8.614046 10.790107  8.626876
ENSMUSG00000000125  3.662672  3.932671  3.929527  6.048455  3.640687  3.713625
ENSMUSG00000000126 10.440646  7.325187  8.010813  8.473362  8.024715  9.196946
ENSMUSG00000000127 10.871825 11.367089 12.520782 12.050519 10.850201 11.332045
ENSMUSG00000000142 11.693639 11.082682 11.650115 11.0

ENSMUSG00000000120  8.632109  9.193715  9.702919 10.873059  7.339519  8.498151
ENSMUSG00000000125  3.976117  9.365695  9.201471  3.829640  7.339519  6.359143
ENSMUSG00000000126 10.707398 10.485416 11.389676  7.314119  7.339519  9.314255
ENSMUSG00000000127 11.454604 12.056996 10.650033 11.479000  7.339519 11.264832
ENSMUSG00000000142 10.498475 12.322336 11.380239 10.483596  7.339519 10.463453
ENSMUSG00000000149 16.237767 13.890631 14.406806 15.724357  7.339519 13.333705
ENSMUSG00000000154  7.219319  6.893026  7.025400 14.804193  7.339519 10.342966
ENSMUSG00000000168 16.581988 14.364091 13.935519 13.828727  7.339519 15.726115
ENSMUSG00000000171 16.241477 12.708406 13.895538 16.042800 23.936579 15.455026
                       V5797     V5798     V5799     V5800     V5801     V5802
ENSMUSG00000000001 15.005640  3.869255 15.206597 14.044734 13.069132 12.293393
ENSMUSG00000000028  9.380831 14.793943  8.042421  9.213867 10.424373  8.735216
ENSMUSG00000000049  6.267883  7.434497 20.548359  9.

ENSMUSG00000000171 15.954362 13.447455 16.515173 16.710028 13.879594 17.528140
                       V5833     V5834     V5835     V5836     V5837     V5838
ENSMUSG00000000001 16.061353 13.217527 13.688209 13.963797 15.638208 14.985445
ENSMUSG00000000028  8.264390 12.583886  8.532602  8.852525 11.002473  8.009514
ENSMUSG00000000049 19.641819 11.865712  5.800314 19.932294  5.905032 20.863470
ENSMUSG00000000058 10.525600  8.231185 11.065385 15.788224 11.625764 10.561178
ENSMUSG00000000085 11.129946 13.626283 12.617745 11.649129 12.052250  9.640951
ENSMUSG00000000088 13.514888 13.447766 11.033000 12.558779 12.773097 15.623672
ENSMUSG00000000120 10.052897 10.653659  8.875377 10.489485 15.369728  9.092273
ENSMUSG00000000125  3.962733 11.453770  6.134295  3.691145  4.668806  4.034059
ENSMUSG00000000126  5.633381  6.294126 11.279855  8.858764

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 10.676526
ENSMUSG00000000149 15.117445 14.232301 14.210363 14.639027 15.935078  3.847454
ENSMUSG00000000154 15.411633  6.492549 13.402835  7.143393 14.788311 17.269073
ENSMUSG00000000168 13.780456 15.955198 13.769508 13.703290 13.948920 16.708050
ENSMUSG00000000171 16.703056 16.378603 15.249903 12.579645 16.037531 17.304264
                       V6043     V6044     V6045     V6046     V6047     V6048
ENSMUSG00000000001 15.178476 15.858573 13.263618 12.659315 11.709125 15.194846
ENSMUSG00000000028  8.741409  8.049983  8.883566  7.915508  9.512608  9.074523
ENSMUSG00000000049 20.240652 21.327856  9.129125  6.754548  8.219989 20.028136
ENSMUSG00000000058 10.102427 10.051162 13.363758 12.181380  9.005309 10.592815
ENSMUSG00000000085 10.510061 11.665962 12.091747

In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      GEO_model_training_tissue_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      5838
[5839,]      5839
[5840,]      5840
[5841,]      5841
[5842,]      5842
[5843,]      5843
[5844,]      5844
[5845,]      5845
[5846,]      5846
[5847,]      5847
[5848,]      5848
[5849,]      5849
[5850,]      5850
[5851,]      5851
[5852,]      5852
[5853,]      5853
[5854,]      5854
[5855,]      5855
[5856,]      5856
[5857,]      5857
[5858,]      5858
[5859,]      5859
[5860,]      5860
[5861,]      5861
[5862,]      5862
[5863,]      5863
[5864,]      5864
[5865,]      5865
[5866,]      5866
[5867,]      5867
[5868,]      5868
[5869,]      5869
[5870,]      5870
[5871,]      5871
[5872,]      5872
[5873,]      5873
[5874,]      5874
[5875,]      5875
[5876,]      5876
[5877,]      5877
[5878,]      5878
[5879,]      5879
[5880,]      5880
[5881,]      5881
[5882,]      5882
[5883,]      5883
[5884,]      5884
[5885,]      5885
[5886,]      5886
[5887,]      5887
[5888,]      5888
[5889,]      5889
[5890,]      5890
[5891,]      5891
[5892,]      5892
[5893,]      5893

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)
print(length(vst.count.mtx.test))

                          V1        V2        V3        V4        V5        V6
ENSMUSG00000000001 15.509603 12.614849 15.519749 14.637853 14.055204 13.449194
ENSMUSG00000000028  8.444428  7.614693 10.420513  9.853543  7.905628  8.286601
ENSMUSG00000000049 20.595033  6.507020  8.077450  7.916962 20.798987  6.163521
ENSMUSG00000000058 10.526181 11.266114  8.766421 11.650309  9.692308 15.196969
ENSMUSG00000000085  9.811443 13.441425 11.548222 12.198297 11.906675 12.810133
ENSMUSG00000000088 16.480555 13.705895 14.766641 14.470710 18.266177 11.941156
ENSMUSG00000000120  9.570225  8.731978  9.042672  9.580352 11.275289  7.829244
ENSMUSG00000000125  4.150751  8.571666 10.636493  4.498008  3.916744  5.114607
ENSMUSG00000000126  7.073363  7.212179  7.601713  7.519445  7.816734 10.184154
ENSMUSG00000000127 11.007871 10.848659 10.921434 10.695664 11.936982 11.026716
ENSMUSG00000000142 10.089229 11.974985 14.516736 11.808314 10.162602 12.502987
ENSMUSG00000000149 14.462026 13.909150 12.471860 12.

ENSMUSG00000000120  8.673436  8.097021  8.534829  7.509745 14.287384  9.950903
ENSMUSG00000000125 10.691888 11.788141  9.107100  8.447599 10.539475  3.795602
ENSMUSG00000000126  7.430537 10.838429  9.501188 11.598598 10.537546  7.411336
ENSMUSG00000000127 10.858316 14.595711 11.648636 10.663087 12.409899 12.113986
ENSMUSG00000000142 11.562486 10.841660 12.306999 10.917297 14.173773 11.560116
ENSMUSG00000000149 14.777297 14.144060 14.153404 14.278217 12.319572 14.565825
ENSMUSG00000000154  5.937776  4.390386  6.871156  6.201857  5.612258 13.509432
ENSMUSG00000000168 14.229457 12.527489 13.048111 13.461766  9.063161 13.037069
ENSMUSG00000000171 13.815139 13.725104 13.049585 13.765870  9.018556 15.051934
                         V43       V44       V45       V46       V47       V48
ENSMUSG00000000001 14.863803 15.551397 15.509006 12.881102 15.083503 12.890134
ENSMUSG00000000028 10.222846  8.291748  9.959167 10.339746  8.465116  8.994184
ENSMUSG00000000049  8.770829 20.313778 19.718144  4.

ENSMUSG00000000168 12.979829 12.311299 13.708315 13.553122 13.405115 14.667265
ENSMUSG00000000171 12.931972 14.804357 16.474337 13.424513 15.704662 15.702937
                         V79       V80       V81       V82       V83       V84
ENSMUSG00000000001 12.582164 12.503303 12.640720 15.584722 12.360589 15.655107
ENSMUSG00000000028  7.948058  9.399669  9.131683  8.711484  9.039425  3.999828
ENSMUSG00000000049  8.154628  8.265493  9.322246 19.485815  7.646570 10.611464
ENSMUSG00000000058 12.562865 13.554198 13.822165 10.170056 13.003293 14.090990
ENSMUSG00000000085 11.816218 13.551972 13.467945 11.023644 12.215782 10.335462
ENSMUSG00000000088 13.873622 18.052361 17.152794 15.238426 16.525558 17.466673
ENSMUSG00000000120  6.316021  7.212504  7.577352 10.595759  8.724745  9.582848
ENSMUSG00000000125  8.244991  3.757661  5.478334  3.866427  5.409441  3.999828
ENSMUSG00000000126 11.836819 12.289934 12.020566  7.005304 11.212689 10.369110
ENSMUSG00000000127 12.502172 11.426191 11.439489 11.

ENSMUSG00000000085 13.377384 11.950847 10.682192  7.102949 11.774735  7.384598
ENSMUSG00000000088 13.748764 17.098015 14.128457 13.834486 17.289888 12.944806
ENSMUSG00000000120  8.850066  8.496348  8.831413  7.477018  9.084898  7.384598
ENSMUSG00000000125  3.836952  3.591454  8.891628 12.960936  3.682280  7.384598
ENSMUSG00000000126  8.656505 10.409246  7.762958  7.648244  8.997958  7.384598
ENSMUSG00000000127 13.637552 11.891918 11.251967 12.333786 10.935605 13.652287
ENSMUSG00000000142 10.697952 10.144489 13.998690 12.951234 11.356774 19.361243
ENSMUSG00000000149 12.747787 15.369466 11.971141 13.568618 14.846623 15.746155
ENSMUSG00000000154  8.593219  7.100625 12.085732  9.844779  8.249874  7.384598
ENSMUSG00000000168 15.018110 16.758113 12.602273 12.613954 17.571565  7.384598
ENSMUSG00000000171 13.386707 16.754295 14.942899 13.933354 17.264503 17.441889
                        V121      V122      V123      V124     V125      V126
ENSMUSG00000000001 14.743406 15.700681 12.025687 15.1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ENSMUSG00000000049 19.478852  5.763824 20.572774  7.133813 21.525146  7.479718
ENSMUSG00000000058 10.542520 15.125527  9.716787 16.386405  9.556601 11.658207
ENSMUSG00000000085 11.144466 11.401344 11.282595 11.616975 11.512550 11.760492
ENSMUSG00000000088 15.207400 15.078313 15.645671 15.657608 17.220631 13.512142
ENSMUSG00000000120 10.301691  9.677793  9.553984  7.918427 11.840053  9.317350
ENSMUSG00000000125  3.897135  6.164323  3.718877  4.027919  3.595966 10.237938
ENSMUSG00000000126  7.916039  8.722351  7.833093  9.958945  8.081855  9.188536
ENSMUSG00000000127 10.956167 10.784105 10.410023 11.222706 10.724274 11.058703
ENSMUSG00000000142  9.131076 11.224198 10.653234 10.205223 10.650188 11.410707
ENSMUSG00000000149 15.383286 13.612863 14.408713 13.138743 15.710893 14.216198
ENSMUSG00000000154 14.430935  8.559255 14.983957  9.932728 16.985580  7.112154
ENSMUSG00000000168 13.642762 16.100075 15.884344 15.713426 14.723052 13.557547
ENSMUSG00000000171 16.153877 15.204114 16.870897 15.

ENSMUSG00000000142 12.182352 12.179352 10.620711 10.863368 11.612872 12.747317
ENSMUSG00000000149 13.770836 12.313222 15.147899 15.483335 11.447212 13.612712
ENSMUSG00000000154  4.082443 12.335417 14.533723 15.315066  5.870485  8.889543
ENSMUSG00000000168 12.303478 13.285839 14.391717 13.945686 13.428784 14.298361
ENSMUSG00000000171 13.145139 14.703322 16.873867 16.427231 12.766314 14.005716
                       V6151     V6152     V6153     V6154     V6155     V6156
ENSMUSG00000000001 13.858021 15.246618 16.147615 12.218419 12.616607 12.027351
ENSMUSG00000000028  7.958547  7.818353  9.908708  8.130574  9.185516  8.191208
ENSMUSG00000000049  6.902419 19.620315  8.675348  6.708652  7.712438  7.298346
ENSMUSG00000000058  8.172349 10.137429  9.677555 11.911226 12.103696 11.625368
ENSMUSG00000000085 11.812582 10.201872 10.016658 12.168463 12.822865 12.606567
ENSMUSG00000000088 10.683461 15.397552 15.412323 15.562836 13.364873 14.834841
ENSMUSG00000000120 11.115419 10.111428  8.275667  9.

ENSMUSG00000000028  8.560351  8.409418  9.036271 10.259639  9.449002  8.446507
ENSMUSG00000000049 20.211237  8.769214  8.300905  5.626945 15.942786 20.510194
ENSMUSG00000000058  9.982508 11.479536 15.008997 16.931557  8.863134 10.360410
ENSMUSG00000000085 11.073369 14.659627 12.548248 11.260648 12.425751 11.461145
ENSMUSG00000000088 14.570020 12.538857 12.230865 18.266406 10.293000 16.570293
ENSMUSG00000000120  9.049313  8.044313 13.542918  5.722315 11.015621  9.569123
ENSMUSG00000000125  3.642590  9.725982  3.687014  3.754442  4.004961  3.970832
ENSMUSG00000000126  7.203020  8.350439 11.639349  8.064013  8.055664  7.923325
ENSMUSG00000000127 10.740708 14.323883 11.496442 11.530918 13.076494 10.997598
ENSMUSG00000000142 10.433214 12.994940 11.477306 11.449988 10.682957 10.528704
ENSMUSG00000000149 15.097677 13.028480 15.399004 14.253483 13.626128 15.533342
ENSMUSG00000000154 14.230383  7.174264  9.539331  6.639184 13.306996 15.168579
ENSMUSG00000000168 14.167091 11.503885 12.647081 18.

ENSMUSG00000000127 15.589120 10.503871 11.089960 10.345661 10.702608 11.832756
ENSMUSG00000000142  6.778799 17.680290 10.962785 11.191592 11.908078 11.466505
ENSMUSG00000000149 10.967143  3.930403 15.496010 14.203558 15.343063 16.280473
ENSMUSG00000000154 12.091635  3.930403 14.355027 10.042429 15.417120 14.020498
ENSMUSG00000000168 15.359550 15.365178 13.718798 12.610474 14.790292 14.249815
ENSMUSG00000000171 16.174523 13.976899 16.139203 13.316781 15.796948 16.058943
                       V6229     V6230     V6231     V6232     V6233     V6234
ENSMUSG00000000001 14.235088 14.044979 13.640425 15.858700 14.963118 12.428384
ENSMUSG00000000028  8.766861  9.106853 12.850584  7.652068  9.765005  8.726017
ENSMUSG00000000049  7.163115  5.258298  9.311808 20.382494  7.823879  8.000192
ENSMUSG00000000058  9.749359 16.215730

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
%%R
GEO_model_training_tissue_detail.vec.train <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[training_indices]
GEO_model_training_tissue_detail.vec.test <-
   GEO_model_training_tissue_detail_vec_tis_of_interest[-training_indices]

# print(GEO_model_training_tissue_detail.vec.train)
print(length(unique(GEO_model_training_tissue_detail.vec.train)))

[1] 26


In [17]:
%%R
saveRDS(GEO_model_training_tissue_detail.vec.train,file=paste(OUT_DIR,"GEO_model_training_tissue_detail_vec_train.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(GEO_model_training_tissue_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
  [1]    9   10   42   58   64   72   76   90   92  104  105  113  123  137  163
 [16]  177  183  186  203  219  244  249  269  300  306  311  331  340  348  362
 [31]  366  368  370  371  374  380  389  418  435  447  454  473  488  516  519
 [46]  522  529  531  542  573  574  579  585  612  617  628  632  653  658  664
 [61]  674  675  677  687  708  711  712  714  765  791  794  799  816  823  824
 [76]  831  834  837  847  856  874  876  886  889  911  921  925  929  930  934
 [91]  944  960  969  993 1002 1021 1032 1034 1041 1047 1050 1053 1064 1070 1075
[106] 1090 1106 1129 1131 1133 1144 1152 1153 1171 1176 1188 1218 1226 1228 1231
[121] 1268 1271 1276 1290 1306 1320 1354 1362 1371 1397 1405 1417 1424 1425 1427
[136] 1430 1439 1473 1489 1497 1522 1532 1538 1540 1542 1549 1564 1569 1579 1580
[151] 1584 1594 1600 1607 1624 1638 1653 1656 1679 1683 1715 1729 1734 1765 1767
[166] 1769 1779 1783 1789 1792 1794 1801 1821 1822 1826 1833 1844 1846 1859 1866
[181] 1869 1870 1875

[256] 2476 2484 2490 2495 2522 2525 2526 2527 2552 2568 2582 2583 2606 2623 2657
[271] 2733 2736 2745 2749 2758 2764 2767 2783 2785 2809 2811 2812 2815 2827 2830
[286] 2845 2853 2855 2862 2865 2866 2869 2878 2888 2898 2927 2928 2932 2938 2942
[301] 2946 2948 2959 2963 2967 2990 2993 3017 3021 3023 3035 3046 3050 3051 3054
[316] 3069 3073 3075 3078 3083 3084 3097 3098 3107 3118 3124 3133 3140 3149 3163
[331] 3190 3200 3214 3221 3228 3230 3238 3244 3261 3262 3264 3269 3274 3278 3288
[346] 3316 3329 3353 3356 3369 3376 3386 3409 3426 3438 3465 3468 3471 3473 3475
[361] 3477 3479 3480 3493 3503 3514 3538 3545 3556 3568 3582 3584 3586 3592 3628
[376] 3643 3679 3685 3690 3697 3702 3708 3734 3740 3746 3757 3770 3777 3801 3823
[391] 3875 3886 3906 3920 3926 3927 3928 3934 3943 3945 3946 3949 3973 3979 3983
[406] 3986 3992 3999 4000 4001 4015 4033 4051 4052 4082 4109 4113 4131 4132 4135
[421] 4140 4149 4155 4160 4166 4175 4182 4199 4200 4217 4236 4247 4259 4270 4277
[436] 4289 4295 4301 4305 43

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
%%R
binary_GEO_model_training_tissue_annotations <- unique(GEO_model_training_tissue_detail.vec)
print(binary_GEO_model_training_tissue_annotations)

In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
print(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ])

                         V1       V2       V3       V4       V5       V6
ENSMUSG00000000568 13.30170 15.56688 14.58060 13.76051 14.58367 14.63848
ENSMUSG00000002102 15.80848 13.72309 14.45077 13.84378 15.96145 13.97942
ENSMUSG00000018286 15.71306 12.87155 13.68762 13.11953 16.12952 13.34630
ENSMUSG00000021024 16.33752 13.35242 13.89909 13.31226 15.30929 12.88821
ENSMUSG00000021178 15.37183 14.05861 13.83790 13.31769 17.05977 13.90289
ENSMUSG00000021737 15.47754 13.11858 12.95268 12.76819 14.54536 12.11729
ENSMUSG00000021832 15.98459 13.67313 13.86128 13.76985 15.41361 12.85739
ENSMUSG00000029440 13.14539 11.73279 11.29995 11.02522 13.50948 11.51235
ENSMUSG00000030591 15.68268 13.20122 14.99129 13.92379 16.34133 13.52438
ENSMUSG00000030603 14.99055 12.98968 14.23627 13.16687 15.51189 13.16419
ENSMUSG00000030751 15.97341 13.14587 13.96520 13.44179 15.35010 13.03819
ENSMUSG00000032869 12.04932 11.81507 12.67638 11.44284 13.82566 12.08281
ENSMUSG00000040850 15.73620 13.27299 14.63198 14.17

ENSMUSG00000068749 11.48142 12.57338 13.10456 11.75127 10.41238 12.28981
ENSMUSG00000069744 11.78994 12.86504 12.93248 13.87514 15.52229 13.56075
                        V43      V44      V45      V46      V47      V48
ENSMUSG00000000568 13.47371 12.34403 12.93931 13.94654 14.02196 12.92395
ENSMUSG00000002102 13.38190 14.90539 15.15211 14.81705 16.05357 15.90423
ENSMUSG00000018286 12.78357 14.77113 15.02740 14.74701 15.83476 14.90946
ENSMUSG00000021024 12.82949 14.69725 14.81884 15.39365 15.94908 14.56785
ENSMUSG00000021178 13.04604 14.62124 15.07031 14.56985 15.98634 14.36589
ENSMUSG00000021737 12.45794 14.36637 14.37290 12.96690 14.76308 13.85032
ENSMUSG00000021832 13.12349 14.92633 14.82862 14.94603 15.41398 14.24772
ENSMUSG00000029440 11.31982 12.28799 12.75264 11.68627 15.06656 11.88383
ENSMUSG00000030591 14.30716 15.13141 15.29101 16.00659 15.97783 14.85654
ENSMUSG00000030603 13.85580 14.71183 15.00032 14.32783 15.74686 14.18794
ENSMUSG00000030751 14.19924 14.67739 14.87227 15.02

ENSMUSG00000040850 14.35419 18.35740 17.75645 16.09858 17.24777 18.98577
ENSMUSG00000068749 11.09783 12.06491 12.74831 12.51896 11.80211 11.77869
ENSMUSG00000069744 11.23762 12.79860 13.45336 12.59362 12.49587 16.48401
                        V85      V86      V87       V88      V89      V90
ENSMUSG00000000568 15.37956 13.83339 13.44148 15.278091 16.08571 13.18588
ENSMUSG00000002102 13.94599 14.68693 13.72000 13.855283 15.23102 14.86742
ENSMUSG00000018286 13.48642 13.88781 13.45103 16.088104 12.94602 14.62283
ENSMUSG00000021024 13.54703 14.53117 13.20604 16.716044 13.47617 14.87788
ENSMUSG00000021178 14.07710 14.24922 13.45646 16.312011 16.48193 14.94991
ENSMUSG00000021737 13.10066 13.39528 12.68519 15.902005 13.73607 14.62423
ENSMUSG00000021832 13.70778 13.80852 13.62682 16.622755 13.24218 15.27870
ENSMUSG00000029440 11.55228 11.87427 11.22445  5.904290 11.68779 14.28084
ENSMUSG00000030591 13.20471 14.79783 13.86942  4.089882 16.45765 15.27418
ENSMUSG00000030603 13.31176 13.53148 13.1

ENSMUSG00000030751 13.96667 13.71481 13.26398 14.99089  9.257916 13.81167
ENSMUSG00000032869 11.38719 11.48034 11.64160 13.25082  9.257916 12.25911
ENSMUSG00000040850 14.70628 15.14381 16.07998 15.80884 16.770514 15.95780
ENSMUSG00000068749 12.19139 11.01408 10.73290 12.91271  9.257916 11.91903
ENSMUSG00000069744 13.24194 11.76941 12.44067 13.33483  9.257916 12.43686
                       V127     V128     V129     V130     V131     V132
ENSMUSG00000000568 12.92983 13.96497 13.15015 12.58740 14.36327 14.94611
ENSMUSG00000002102 15.67010 16.20022 13.20430 14.19986 15.35659 13.83847
ENSMUSG00000018286 14.12363 15.39171 13.60120 13.49000 14.68551 13.49564
ENSMUSG00000021024 13.31373 15.29232 14.08822 13.44354 14.17938 13.52538
ENSMUSG00000021178 14.13588 15.33079 12.84334 13.32260 14.79431 13.97715
ENSMUSG00000021737 12.19545 15.53669 13.66837 12.78623 13.48950 13.40349
ENSMUSG00000021832 12.39915 15.21866 13.23703 13.46444 14.50245 14.53244
ENSMUSG00000029440 12.65038 12.32675 11.46890 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

ENSMUSG00000018286 14.52086 15.25056 14.68684 19.06261 13.27329 14.68070
ENSMUSG00000021024 15.29927 15.22292 15.32789 20.06606 13.02900 14.57204
ENSMUSG00000021178 14.43128 14.73869 14.99671 16.26717 14.07492 15.28602
ENSMUSG00000021737 14.02582 15.63391 14.41279 17.68441 13.46206 14.30428
ENSMUSG00000021832 15.15392 15.53430 15.59906 19.74667 13.96948 14.10152
ENSMUSG00000029440 13.23707 13.72388 13.61995 13.34586 11.60541 12.41386
ENSMUSG00000030591 17.21621 15.48800 14.31866 14.35997 13.34202 15.07219
ENSMUSG00000030603 15.52181 14.30989 14.55536 15.48261 13.37102 14.48711
ENSMUSG00000030751 13.25295 15.21672 15.10779 16.74259 13.30724 14.08941
ENSMUSG00000032869 12.05160 12.64790 12.81477 16.76681 11.45764 12.98230
ENSMUSG00000040850 14.95362 15.63564 15.92141 15.68723 14.02628 15.63809
ENSMUSG00000068749 12.25800 10.64669 12.28318 16.69212 11.91816 11.67126
ENSMUSG00000069744 13.54444 11.88074 12.44337 15.28999 12.19993 12.81603
                      V5971    V5972    V5973    V5

ENSMUSG00000002102 14.95303 15.36417 14.17625 15.68904 14.94792 14.46554
ENSMUSG00000018286 13.88665 14.26764 13.45955 14.68630 14.76725 15.77729
ENSMUSG00000021024 13.54432 14.34226 13.14212 16.27467 14.06477 15.86211
ENSMUSG00000021178 14.67372 15.37417 13.88964 15.41606 14.82510 15.62371
ENSMUSG00000021737 13.80852 13.70405 13.06456 15.93012 14.22752 14.72883
ENSMUSG00000021832 14.66819 13.78979 13.59673 15.73432 14.71616 15.31061
ENSMUSG00000029440 11.69515 12.12082 11.54611 12.64958 13.50437 12.97551
ENSMUSG00000030591 14.25736 14.39422 13.43509 14.80029 15.11141 16.93802
ENSMUSG00000030603 14.13588 14.04795 13.17770 13.94141 14.76324 17.29017
ENSMUSG00000030751 13.32918 13.66443 13.12615 14.91176 14.15395 15.98331
ENSMUSG00000032869 11.52666 12.91005 11.95590 11.39845 12.66923 12.17019
ENSMUSG00000040850 14.82769 11.40797 13.75948 17.56946 16.19216 16.07476
ENSMUSG00000068749 12.26411 13.01786 11.51562 13.00672 12.08693 12.88971
ENSMUSG00000069744 12.68123 13.01452 11.36805 13.31

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 16.884262 15.249593 15.05116 14.074681 14.19608 14.30767
ENSMUSG00000068749 14.038392 13.179263 11.42010  9.351357 13.19282 10.91295
ENSMUSG00000069744 14.820511 14.096629 12.49744  9.351357 14.03298 11.86963
                      V6241    V6242    V6243    V6244    V6245    V6246
ENSMUSG00000000568 12.48007 14.17339 13.21288 11.42176 14.95421 13.82257
ENSMUSG00000002102 13.62904 16.21438 13.46498 15.04705 15.28822 14.70010
ENSMUSG00000018286 14.00379 16.01708 13.54029 14.63809 16.35022 14.05825
ENSMUSG00000021024 14.54587 16.07135 12.84489 14.05823 15.27200 14.92060
ENSMUSG00000021178 13.20093 16.09564 13.39995 14.87839 15.39701 15.05802
ENSMUSG00000021737 13.74213 15.07959 12.93544 14.19836 13.21297 13.98272
ENSMUSG00000021832 13.94422 15.34997 13.69119 14.21541 15.38920 14.65072
ENSMUSG00000029440 10.40328 14.91819 11.89440 12.96418 10.25620 12.45114
ENSMUSG00000030591 13.97380 16.03303 13.82933 15.35280 14.05626 14.48588
ENSMUSG00000030603 12.49871 16.01391 13.12960 14.80175 15.01

ENSMUSG00000030751  9.918272 12.714397 15.02070 13.91494 10.71095  8.412901
ENSMUSG00000032869  9.918272  3.918626 11.98370 11.70108 13.98995  8.797332
ENSMUSG00000040850 14.190413 15.092763 17.79251 17.26316 15.49977 12.526367
ENSMUSG00000068749  9.918272 10.467822 11.53044 11.56691 13.95678  8.491079
ENSMUSG00000069744  9.918272 12.816496 11.40877 12.57192 15.37286  9.532581
                       V6283     V6284    V6285    V6286    V6287    V6288
ENSMUSG00000000568 13.247895 24.243015 13.21470 13.22305 12.53093 13.45742
ENSMUSG00000002102 12.673599 22.271301 13.99817 14.11287 14.46900 16.46140
ENSMUSG00000018286 13.937142  9.388403 13.18772 13.94689 14.00430 18.21405
ENSMUSG00000021024 13.906433 20.723623 13.08419 13.61298 14.81197 18.66801
ENSMUSG00000021178 13.811191  9.388403 13.66346 13.81915 14.54958 14.71393
ENSMUSG00000021737 14.319774  9.388403 12.77278 13.88360 13.99775 14.84141
ENSMUSG00000021832 13.913911  9.388403 13.43884 12.69506 14.78561 15.95304
ENSMUSG00000029440  

In [22]:
%%R
test <- prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]))
print(test)

Standard deviations (1, .., p=17):
 [1] 4.2900621 3.0830072 2.6153031 2.2194112 2.0171399 1.7432980 1.6601397
 [8] 1.3917216 1.3102652 1.2516235 1.2083681 1.1906495 1.1120174 1.0226376
[15] 0.9912435 0.9227105 0.8866596

Rotation (n x k) = (17 x 17):
                           PC1         PC2          PC3         PC4
ENSMUSG00000000568 -0.05764346 -0.44957687  0.114515261 -0.71776192
ENSMUSG00000002102  0.24743754 -0.15808074 -0.566024642 -0.03597337
ENSMUSG00000018286  0.34019131  0.15886328  0.015793667 -0.01605893
ENSMUSG00000021024  0.32164986  0.17526531 -0.002295924 -0.07083264
ENSMUSG00000021178  0.25447907  0.14991952  0.020724286 -0.14943801
ENSMUSG00000021737  0.26326532  0.16436359  0.015026283 -0.12828858
ENSMUSG00000021832  0.24076094  0.19370837  0.074578403 -0.17994911
ENSMUSG00000029440  0.17632898 -0.39687203  0.130401050  0.13821481
ENSMUSG00000030591  0.30495490 -0.39918578  0.225629547  0.12446440
ENSMUSG00000030603  0.26611427 -0.39084714  0.193717318  0.13090342
E

In [23]:
%%R
test$x[,1]

           V1            V2            V3            V4            V5 
-4.363800e+00 -2.803478e+00  6.484980e-01 -1.412667e+01 -6.785720e+00 
           V6            V7            V8            V9           V10 
-4.920129e-02 -3.878238e+00  1.821015e+00  2.591119e+00  3.156022e+00 
          V11           V12           V13           V14           V15 
-8.360156e+00  3.496878e+00 -6.449246e-02 -2.180130e+00 -6.376654e-01 
          V16           V17           V18           V19           V20 
 1.191947e+00  2.166660e+00  8.450806e+00 -1.485854e+00 -6.904557e+00 
          V21           V22           V23           V24           V25 
 5.042111e+00  4.904570e+00  4.150228e+00  3.282066e+00  1.130533e+01 
          V26           V27           V28           V29           V30 
 7.652237e-01  4.854201e+00  3.044039e+00 -3.223128e+00 -1.787215e+00 
          V31           V32           V33           V34           V35 
-1.792952e+00 -2.222686e+00 -1.856970e+00  3.844920e+00  7.290125e+00 
      

         V576          V577          V578          V579          V580 
 3.335891e+00 -2.114143e+00 -1.021555e+00 -3.162832e+00 -3.659449e+00 
         V581          V582          V583          V584          V585 
 1.983517e+00 -9.694099e-01 -7.128844e+00 -6.241056e-01 -9.924745e-01 
         V586          V587          V588          V589          V590 
-9.071599e+00  1.942070e+00 -2.711994e+00 -1.565428e-01 -1.152987e+00 
         V591          V592          V593          V594          V595 
-4.506703e+00  3.945343e+00 -1.396881e+00 -2.441358e+00  1.477779e+00 
         V596          V597          V598          V599          V600 
 4.683673e+00  1.269655e+00  3.759494e+00 -3.652940e+00  4.619542e+00 
         V601          V602          V603          V604          V605 
-1.161893e+01  3.738167e+00  2.595610e+00  2.251599e+00 -1.415764e+00 
         V606          V607          V608          V609          V610 
-9.289966e+00  2.376785e+00  3.452972e+00 -1.904214e+00 -4.633501e+00 
      

        V1151         V1152         V1153         V1154         V1155 
-1.091309e+00  4.608256e-01  1.274306e+01 -2.226239e+00 -1.572549e+00 
        V1156         V1157         V1158         V1159         V1160 
 7.595502e-01 -3.283114e+00 -3.550815e+00  1.051889e+00  3.644440e+00 
        V1161         V1162         V1163         V1164         V1165 
 3.782762e+00 -1.685930e+00  3.465214e+00  8.624204e-01 -3.886371e+00 
        V1166         V1167         V1168         V1169         V1170 
-9.026817e-01 -1.731494e+00  3.237760e+00  6.456755e-01 -3.957457e+00 
        V1171         V1172         V1173         V1174         V1175 
 5.006056e+00  6.062025e+00  1.380794e+00 -1.367920e+00  4.807806e+00 
        V1176         V1177         V1178         V1179         V1180 
-2.080230e+00  3.805349e+00  4.592211e+00 -3.000093e+00  3.244130e+00 
        V1181         V1182         V1183         V1184         V1185 
-3.195483e+00  3.854535e+00  2.058715e+00 -2.132423e+00 -1.235739e+00 
      

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



        V1846         V1847         V1848         V1849         V1850 
-1.128290e+00 -1.821409e+00  1.584980e+00 -9.712556e-01  4.301066e+00 
        V1851         V1852         V1853         V1854         V1855 
 4.202217e+00 -3.232384e+00 -3.213404e+00 -3.914912e+00  1.643780e+00 
        V1856         V1857         V1858         V1859         V1860 
 2.995399e+00 -2.701697e+00 -3.943029e+00 -3.020242e+00 -2.510420e+00 
        V1861         V1862         V1863         V1864         V1865 
-2.055631e+00 -5.113603e+00  1.971955e+00  2.618918e+00  2.494798e+00 
        V1866         V1867         V1868         V1869         V1870 
-3.732654e+00 -9.390572e-01 -9.564496e-01  4.878210e+00 -1.913311e+00 
        V1871         V1872         V1873         V1874         V1875 
 1.818963e+00 -3.238505e+00 -1.592845e+00  2.581459e+00 -7.979565e-01 
        V1876         V1877         V1878         V1879         V1880 
-2.770020e+00 -2.645772e+00 -1.818632e+00 -2.519407e+00 -3.368430e+00 
      

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



-1.443721e+00  1.187127e+00 -7.728209e+00 -2.029622e+00 -3.380234e+00 
        V3866         V3867         V3868         V3869         V3870 
-2.186608e+00  1.031576e+00  2.735811e+00  2.479711e+00 -7.697697e+00 
        V3871         V3872         V3873         V3874         V3875 
 1.577042e+00  2.822372e+00 -2.159715e+00  3.314796e+00  1.663678e+00 
        V3876         V3877         V3878         V3879         V3880 
 4.607781e+00 -3.986023e+00  1.910393e+00  3.862950e+00 -3.232307e+00 
        V3881         V3882         V3883         V3884         V3885 
-2.544403e+00  2.661075e+00 -6.209850e+00 -7.007956e+00  6.489835e-01 
        V3886         V3887         V3888         V3889         V3890 
-3.410903e-01  4.415241e+00  4.449486e+00 -2.366912e+00 -1.592684e+00 
        V3891         V3892         V3893         V3894         V3895 
-2.192176e+00 -3.661637e+00 -1.708586e+00  7.669224e+00  6.171515e-01 
        V3896         V3897         V3898         V3899         V3900 
 2.329

-1.291957e+00  4.840053e+00  4.133206e+00 -1.245930e+01 -4.939971e+00 
        V4441         V4442         V4443         V4444         V4445 
-2.657031e+00  5.230987e+00  4.138216e+00  2.004281e+00 -3.452464e+00 
        V4446         V4447         V4448         V4449         V4450 
-4.053738e+00 -1.209570e+00 -1.671622e+00 -4.133489e+00  3.550926e+00 
        V4451         V4452         V4453         V4454         V4455 
 3.870781e+00  8.353084e+00  5.203850e+00 -9.870742e-01  3.430659e+00 
        V4456         V4457         V4458         V4459         V4460 
 4.044296e+00  6.037367e+00  5.237806e-01 -4.661738e+00  4.661828e+00 
        V4461         V4462         V4463         V4464         V4465 
 2.095309e+00  3.693829e+00 -2.082831e+00  2.722177e+00  3.280122e+00 
        V4466         V4467         V4468         V4469         V4470 
-2.855127e+00  9.708551e-01 -1.145328e+00 -1.224352e+00  4.113270e+00 
        V4471         V4472         V4473         V4474         V4475 
 6.268

 1.092552e+00  9.405591e-01 -9.878137e-01 -1.017055e+00  2.693177e+00 
        V5016         V5017         V5018         V5019         V5020 
-2.641236e+00  4.452071e+00  1.590957e-02 -1.619549e-01 -2.394332e+00 
        V5021         V5022         V5023         V5024         V5025 
-4.510948e-01  2.815441e+00 -4.539206e+00 -4.197330e+00  2.306564e+00 
        V5026         V5027         V5028         V5029         V5030 
 4.657213e+00 -1.417145e+00  2.786906e+00  3.341332e+00  1.100349e+00 
        V5031         V5032         V5033         V5034         V5035 
-2.123595e+00  5.499997e-01  1.084959e-01  1.652680e+00 -2.204380e-01 
        V5036         V5037         V5038         V5039         V5040 
 1.713533e+00  3.759790e+00  3.452684e+00 -6.967499e-01 -1.762946e+00 
        V5041         V5042         V5043         V5044         V5045 
-6.710367e-01  4.788404e-01 -1.392801e+00 -2.903834e+00  5.810523e+00 
        V5046         V5047         V5048         V5049         V5050 
 4.713

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
# %%R

# all_rotations_df <- data.frame()

# n_rxns <- length(rxns)

# for(rxn_id_idx in seq(1: n_rxns)){
#     rxn_id <- rxns[rxn_id_idx]
#     print(rxn_id)
#     rxn_pca <- prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
#     rotation_df <- as.data.frame(rxn_pca$rotation)
#     rotation_df$RXN_ID <- rxn_id
#     all_rotations_df <- bind_rows(all_rotations_df, rotation_df)
# }

# write.csv(all_rotations_df, file=paste(OUT_DIR, "all_pca_score.csv", seq="\t"), row.names = TRUE)

In [22]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,".Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 7856
[1] "Processed 100 of 7856 reactions (1.3%)..."
[1] "Processed 200 of 7856 reactions (2.6%)..."
[1] "Processed 300 of 7856 reactions (3.8%)..."
[1] "Processed 400 of 7856 reactions (5.1%)..."
[1] "Processed 500 of 7856 reactions (6.4%)..."
[1] "Processed 600 of 7856 reactions (7.7%)..."
[1] "Processed 700 of 7856 reactions (8.9%)..."
[1] "Processed 800 of 7856 reactions (10.2%)..."
[1] "Processed 900 of 7856 reactions (11.5%)..."
[1] "Processed 1000 of 7856 reactions (12.7%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 7856 reactions (12.7%)..."
[1] "Processed 1100 of 7856 reactions (14%)..."
[1] "Processed 1200 of 7856 reactions (15.3%)..."
[1] "Processed 1300 of 7856 reactions (16.6%)..."
[1] "Processed 1400 of 7856 reactions (17.8%)..."
[1] "Processed 1500 of 7856 reactions (19.1%)..."
[1] "Processed 1600 of 7856 reactions (20.4%)..."
[1] "Processed 1700 of 7856 reactions (21.7%)..."
[1] "Processed 1800 of 7856 reactions (22.9%)..."
[1] "Processed 1900 of 785

In [23]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

            used   (Mb) gc trigger   (Mb)  max used   (Mb)
Ncells   6961050  371.8   13280791  709.3  13280791  709.3
Vcells 231945402 1769.7  384120317 2930.7 270961072 2067.3


In [24]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train.Rds", sep = ""))

In [25]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

  [1]    9   10   42   58   64   72   76   90   92  104  105  113  123  137  163
 [16]  177  183  186  203  219  244  249  269  300  306  311  331  340  348  362
 [31]  366  368  370  371  374  380  389  418  435  447  454  473  488  516  519
 [46]  522  529  531  542  573  574  579  585  612  617  628  632  653  658  664
 [61]  674  675  677  687  708  711  712  714  765  791  794  799  816  823  824
 [76]  831  834  837  847  856  874  876  886  889  911  921  925  929  930  934
 [91]  944  960  969  993 1002 1021 1032 1034 1041 1047 1050 1053 1064 1070 1075
[106] 1090 1106 1129 1131 1133 1144 1152 1153 1171 1176 1188 1218 1226 1228 1231
[121] 1268 1271 1276 1290 1306 1320 1354 1362 1371 1397 1405 1417 1424 1425 1427
[136] 1430 1439 1473 1489 1497 1522 1532 1538 1540 1542 1549 1564 1569 1579 1580
[151] 1584 1594 1600 1607 1624 1638 1653 1656 1679 1683 1715 1729 1734 1765 1767
[166] 1769 1779 1783 1789 1792 1794 1801 1821 1822 1826 1833 1844 1846 1859 1866
[181] 1869 1870 1875 1890 18

[241] 2378 2381 2383 2387 2397 2414 2416 2426 2428 2439 2441 2444 2461 2474 2475
[256] 2476 2484 2490 2495 2522 2525 2526 2527 2552 2568 2582 2583 2606 2623 2657
[271] 2733 2736 2745 2749 2758 2764 2767 2783 2785 2809 2811 2812 2815 2827 2830
[286] 2845 2853 2855 2862 2865 2866 2869 2878 2888 2898 2927 2928 2932 2938 2942
[301] 2946 2948 2959 2963 2967 2990 2993 3017 3021 3023 3035 3046 3050 3051 3054
[316] 3069 3073 3075 3078 3083 3084 3097 3098 3107 3118 3124 3133 3140 3149 3163
[331] 3190 3200 3214 3221 3228 3230 3238 3244 3261 3262 3264 3269 3274 3278 3288
[346] 3316 3329 3353 3356 3369 3376 3386 3409 3426 3438 3465 3468 3471 3473 3475
[361] 3477 3479 3480 3493 3503 3514 3538 3545 3556 3568 3582 3584 3586 3592 3628
[376] 3643 3679 3685 3690 3697 3702 3708 3734 3740 3746 3757 3770 3777 3801 3823
[391] 3875 3886 3906 3920 3926 3927 3928 3934 3943 3945 3946 3949 3973 3979 3983
[406] 3986 3992 3999 4000 4001 4015 4033 4051 4052 4082 4109 4113 4131 4132 4135
[421] 4140 4149 4155 4160 41

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 3687 3715 3718 3732 3756 3785 3786
[361] 3795 3826 3828 3829 3835 3844 3846 3847 3849 3856 3869 3893 3894 3898 3922
[376] 3930 3936 3942 3947 3950 3954 3963 3964 3966 4010 4012 4018 4032 4037 4044
[391] 4045 4056 4062 4069 4073 4078 4094 4115 4141 4142 4153 4157 4167 4173 4177
[406] 4215 4228 4245 4246 4262 4265 4276 4278 4312 4330 4337 4363 4365 4370 4371
[421] 4374 4378 4381 4389 4406 4414 4418 4425 4445 4448 4457 4468 4477 4496 4505
[436] 4520 4539 4564 4583 4586 4596 4597 4604 4621 4639 4643 4651 4676 4684 4715
[451] 4719 4722 4740 4745 4774 4775 4780 4808 4829 4848 4877 4885 4887 4905 4910
[466] 4921 4928 4936 4941 4942 4947 4968 4971 4974 4975 4982 4984 5006 5017 5021
[481] 5028 5036 5040 5057 5069 5073 5102 5116 5122 5123 5132 5135 5140 5161 5172
[496] 5191 5202 5207 5211 5214 5222 5226 5233 5241 5242 5243 5252 5262 5276 5281
[511] 5292 5296 5304 5318 5322 5329 5330 5360 5380 5392 5393 5399 5401 5403 5430
[526] 5432 5433 5444 5460 5465 5467 5468 5469 5470 5480 5495 5496 5497 55

In [26]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0
   
   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] 
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] 
      
      GEO_model_training_tissue_detail.vec.train.cv_train <-
         GEO_model_training_tissue_detail.vec.train[-cur_cv_fold_indices] 
      GEO_model_training_tissue_detail.vec.train.cv_test <-
         GEO_model_training_tissue_detail.vec.train[cur_cv_fold_indices] 
       
      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]] <-
            (GEO_model_training_tissue_detail.vec.train.cv_test == tissue_annotation)
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = GEO_model_training_tissue_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            GEO_model_training_tissue_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_GEO_model_training_tissue_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         tab <- table(cur_rxn_knn_calls,
                      binary_GEO_model_training_tissue_detail_vec.test.cv_test_list[[tissue_annotation]])

         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_GEO_model_training_tissue_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-6798748 : Last ARI =  0.932812834238019 : Last ECOUNT =  55 : Last Lung MISCLASS =  0.00333864161004714 : Last Uterus MISCLASS =  0.00111465452666072 . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9840884 : Last ARI =  0.563438784977034 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.032122719008372 : Last Uterus MISCLASS =  0.00604236659974005 . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2002466 : Last ARI =  0.568827950627364 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0254400985329623 : Last Uterus MISCLASS =  0.00556441448870959 . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-2247925 : Last ARI =  0.195308131180437 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0722016850943843 : Last Uterus MISCLASS =  0.0144745662935967 . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9016782 : Last ARI =  0.195257413093143 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0722016850943843 : Last Uterus MISCLASS =  0.0144745662935967 . Now  99.4 % remai

[1] "Last RXN_ID =  R-MMU-8850527 : Last ARI =  0.748277783843302 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.0108176952281766 : Last Uterus MISCLASS =  0.00413558425819044 . Now  94.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1604732 : Last ARI =  0.530174308619139 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0602738841455647 : Last Uterus MISCLASS =  0.0122447444624807 . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213200 : Last ARI =  0.801515150356627 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0264053970727269 : Last Uterus MISCLASS =  0.00540339863506811 . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8948234 : Last ARI =  0.939679872851746 : Last ECOUNT =  17 : Last Lung MISCLASS =  0.00286422566218848 : Last Uterus MISCLASS =  0.000953392772351891 . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-913529 : Last ARI =  0.0378828987377555 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0628249653275887 : Last Uterus MISCLASS =  0.0146355716340012 . Now  94.3 % rem

[1] "Last RXN_ID =  R-MMU-158484 : Last ARI =  0.860403943350665 : Last ECOUNT =  21 : Last Lung MISCLASS =  0.00540896735164219 : Last Uterus MISCLASS =  0.00127262761432307 . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-450595 : Last ARI =  0.251585179678798 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0537641707902743 : Last Uterus MISCLASS =  0.00985949321369433 . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-5578744 : Last ARI =  0.85440872596897 : Last ECOUNT =  21 : Last Lung MISCLASS =  0.00572516671454706 : Last Uterus MISCLASS =  0.00206931390242168 . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-70286 : Last ARI =  0.260058359988702 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0672752171101014 : Last Uterus MISCLASS =  0.0130487796058606 . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852136 : Last ARI =  0.706997051649048 : Last ECOUNT =  6 : Last Lung MISCLASS =  0.013203454409552 : Last Uterus MISCLASS =  0.00190754423085637 . Now  89.2 % remaini

[1] "Last RXN_ID =  R-MMU-8982020 : Last ARI =  0.882724284664775 : Last ECOUNT =  9 : Last Lung MISCLASS =  0.00524542312482744 : Last Uterus MISCLASS =  0.00191083772345704 . Now  84.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-165162 : Last ARI =  0.241958734491862 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0580443469146471 : Last Uterus MISCLASS =  0.0112910985486198 . Now  84.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9693250 : Last ARI =  0.28644151451443 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.054710991912673 : Last Uterus MISCLASS =  0.00954303989791705 . Now  84.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-170844 : Last ARI =  0.363125929273553 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0516842547240394 : Last Uterus MISCLASS =  0.0136761070537485 . Now  84.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-2160932 : Last ARI =  0.819321336577609 : Last ECOUNT =  7 : Last Lung MISCLASS =  0.00429557861700273 : Last Uterus MISCLASS =  0.000635930047694755 . Now  84.1 % remaini

[1] "Last RXN_ID =  R-MMU-382613 : Last ARI =  0.596848378980403 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.035939572334972 : Last Uterus MISCLASS =  0.00492923989322586 . Now  79.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2161792 : Last ARI =  0.708217627987961 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.0170137408853821 : Last Uterus MISCLASS =  0.00906305284169352 . Now  79.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-983194 : Last ARI =  0.792423578560121 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0165443978074578 : Last Uterus MISCLASS =  0.00493202383810377 . Now  79.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696220 : Last ARI =  0.135521717998938 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0655117245618657 : Last Uterus MISCLASS =  0.0141545695276859 . Now  79.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-198746 : Last ARI =  0.69802095644529 : Last ECOUNT =  4 : Last Lung MISCLASS =  0.0125652362521072 : Last Uterus MISCLASS =  0.00636059363630666 . Now  79 % remaining...

...

[1] "Last RXN_ID =  R-MMU-443777 : Last ARI =  0.356260431526036 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0601118636859253 : Last Uterus MISCLASS =  0.013203201218102 . Now  23.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139941 : Last ARI =  0.290482500783985 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0644127667759985 : Last Uterus MISCLASS =  0.0114521127795102 . Now  23.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-416683 : Last ARI =  0.372599790334903 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0217829759181925 : Last Uterus MISCLASS =  0.0133548380779229 . Now  23.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5159250 : Last ARI =  0.0451140718866535 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0704477989368182 : Last Uterus MISCLASS =  0.0151071980073062 . Now  23.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-879575 : Last ARI =  0.631202434671597 : Last ECOUNT =  3 : Last Lung MISCLASS =  0.0491379868524495 : Last Uterus MISCLASS =  0.011769330376434 . Now  23 % remaining..."
[1

[1] "Last RXN_ID =  R-MMU-110360 : Last ARI =  0.0394162081982726 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0686989953509381 : Last Uterus MISCLASS =  0.0163778019663792 . Now  18.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-73931 : Last ARI =  0.0900179327224663 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0666337094668582 : Last Uterus MISCLASS =  0.012412342440761 . Now  18.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-443774 : Last ARI =  0.159983528064741 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.066952186422504 : Last Uterus MISCLASS =  0.0144727836861422 . Now  18.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8985591 : Last ARI =  0.199018366999372 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0632781021514339 : Last Uterus MISCLASS =  0.0139983911103797 . Now  18 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029468 : Last ARI =  0.0543068303431334 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0698141280086869 : Last Uterus MISCLASS =  0.0130361208280911 . Now  17.9 % remaining..."


[1] "Last RXN_ID =  R-MMU-74733 : Last ARI =  0.112213974615456 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0690268175266849 : Last Uterus MISCLASS =  0.0136763554000663 . Now  13.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-2730896 : Last ARI =  0.130282133180313 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0656894602547522 : Last Uterus MISCLASS =  0.0147892329642807 . Now  13.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-73620 : Last ARI =  0.501442494362596 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0653577979810553 : Last Uterus MISCLASS =  0.0171772745989352 . Now  13.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-5358484 : Last ARI =  0.211466730940487 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0532728157275724 : Last Uterus MISCLASS =  0.0128784073267976 . Now  12.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-140842 : Last ARI =  0.499599479804111 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0534274831945566 : Last Uterus MISCLASS =  0.0133591506620015 . Now  12.8 % remaining..."


[1] "Last RXN_ID =  R-MMU-1655453 : Last ARI =  0.244853729836229 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0683762098334221 : Last Uterus MISCLASS =  0.0138330618804986 . Now  8.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8870465 : Last ARI =  0.154189073471239 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0702938490454132 : Last Uterus MISCLASS =  0.0132009155347022 . Now  8.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-1678944 : Last ARI =  0.0360416037479422 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0629859489493866 : Last Uterus MISCLASS =  0.0138396319614873 . Now  8 % remaining..."
[1] "Last RXN_ID =  R-MMU-983702 : Last ARI =  0.0500686457690428 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0494620196504288 : Last Uterus MISCLASS =  0.0147907551509552 . Now  7.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-212005 : Last ARI =  0.646035920806795 : Last ECOUNT =  5 : Last Lung MISCLASS =  0.00747727585970146 : Last Uterus MISCLASS =  0.0090650821611297 . Now  7.7 % remaining..."
[

[1] "Last RXN_ID =  R-MMU-6811508 : Last ARI =  0.0409642138718676 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0739472080810088 : Last Uterus MISCLASS =  0.0125662537170864 . Now  3 % remaining..."
[1] "Last RXN_ID =  R-MMU-450325 : Last ARI =  0.065320043254435 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0644061934842383 : Last Uterus MISCLASS =  0.0130422039068059 . Now  2.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-3446785 : Last ARI =  0.169652680647349 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0602660498913872 : Last Uterus MISCLASS =  0.014152043565506 . Now  2.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5688294 : Last ARI =  0.0452873369986567 : Last ECOUNT =  1 : Last Lung MISCLASS =  0.0640824105476331 : Last Uterus MISCLASS =  0.015105933803501 . Now  2.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-112033 : Last ARI =  0.290317001780479 : Last ECOUNT =  2 : Last Lung MISCLASS =  0.0650527561817868 : Last Uterus MISCLASS =  0.0152641608631893 . Now  2.5 % remaining..."
[1] 

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df.Rds", sep = ""))

                                   RXN_ID     MISCLASS         ARI ECOUNT
R-MMU-450466.Liver           R-MMU-450466 0.0211485608 0.878641413     17
R-MMU-450466.Brain           R-MMU-450551 0.0287865913 0.470006319      4
R-MMU-450466.Intestine       R-MMU-450580 0.0074795607 0.470277781      4
R-MMU-450466.Lung             R-MMU-72103 0.0055687126 0.865871344     23
R-MMU-450466.Muscle         R-MMU-9770119 0.0058838944 0.871532237     33
R-MMU-450466.Testes         R-MMU-9770141 0.0033419431 0.861646735     61
R-MMU-450466.Heart          R-MMU-9770142 0.0087483797 0.863223516     67
R-MMU-450466.Skin           R-MMU-9770145 0.0077934663 0.861596210     75
R-MMU-450466.Kidney         R-MMU-9770236 0.0124052338 0.857053790     59
R-MMU-450466.Uterus         R-MMU-6798748 0.0007959276 0.932812834     55
R-MMU-450466.Sperm          R-MMU-6800434 0.0033429542 0.941947169     69
R-MMU-450466.Eye            R-MMU-6791218 0.0063641322 0.845706873     30
R-MMU-450466.Adipose        R-MMU-6791

R-MMU-9770119.Testes        R-MMU-5652009 0.0028647416 0.836931508     15
R-MMU-9770119.Heart         R-MMU-5652151 0.0111326255 0.717832153      7
R-MMU-9770119.Skin          R-MMU-5653754 0.0044555778 0.710612365      9
R-MMU-9770119.Kidney        R-MMU-5653756 0.0103303617 0.677149693      8
R-MMU-9770119.Uterus        R-MMU-5653766 0.0007961800 0.793086990     10
R-MMU-9770119.Sperm         R-MMU-5653770 0.0020675458 0.793339447     10
R-MMU-9770119.Eye           R-MMU-5653780 0.0049289915 0.726217999      7
R-MMU-9770119.Adipose       R-MMU-5653786 0.0087402794 0.726217999      7
R-MMU-9770119.Adrenal       R-MMU-5653838 0.0000000000 0.781692678     11
R-MMU-9770119.Ovary         R-MMU-5654985 0.0069980475 0.791691049      9
R-MMU-9770119.Olfactory     R-MMU-5654986 0.0004772023 0.622867969      7
R-MMU-9770119.Pancreas      R-MMU-5654989 0.0074737373 0.791691049      9
R-MMU-9770119.Bone          R-MMU-5655466 0.0022267815 0.564858060      6
R-MMU-9770119.Nerve         R-MMU-5655

R-MMU-9770236.Eye           R-MMU-5675456 0.0052456859 0.679859266      5
R-MMU-9770236.Adipose         R-MMU-71050 0.0082661255 0.255000917      2
R-MMU-9770236.Adrenal         R-MMU-73945 0.0003184721 0.348632431      3
R-MMU-9770236.Ovary           R-MMU-75244 0.0050917715 0.255465477      2
R-MMU-9770236.Olfactory       R-MMU-83586 0.0003177127 0.348632196      3
R-MMU-9770236.Pancreas        R-MMU-83650 0.0058821312 0.348362846      3
R-MMU-9770236.Bone          R-MMU-9018794 0.0014313537 0.894453383     23
R-MMU-9770236.Nerve         R-MMU-6798739 0.0004769475 0.925175816     29
R-MMU-9770236.Stomach       R-MMU-6799350 0.0017475515 0.915819448     49
R-MMU-9770236.MOE            R-MMU-888589 0.0001597444 0.844757107      7
R-MMU-9770236.Spinal cord    R-MMU-917744 0.0042922956 0.845351271      7
R-MMU-9770236.Aorta         R-MMU-9664156 0.0019077990 0.825402034      6
R-MMU-9770236.Mammary gland R-MMU-6785821 0.0019103306 0.452019374      3
R-MMU-9770236.Tendon        R-MMU-6786

R-MMU-6791222.Bone          R-MMU-3928614 0.0020677917 0.894132877      8
R-MMU-6791222.Nerve         R-MMU-3928615 0.0006351738 0.870462575      7
R-MMU-6791222.Stomach       R-MMU-3928617 0.0014300888 0.879005944      7
R-MMU-6791222.MOE           R-MMU-3928624 0.0000000000 0.847651234      6
R-MMU-6791222.Spinal cord   R-MMU-3928625 0.0044505170 0.896291802     13
R-MMU-6791222.Aorta         R-MMU-3928627 0.0014305943 0.871292666      7
R-MMU-6791222.Mammary gland R-MMU-3928628 0.0006361848 0.927277526     16
R-MMU-6791222.Tendon        R-MMU-3928630 0.0012723721 0.440722167      2
R-MMU-6791222.VNO           R-MMU-3928631 0.0001589825 0.900085899      9
R-MMU-1605595.Liver         R-MMU-3928633 0.1095809791 0.900938788     10
R-MMU-1605595.Brain         R-MMU-3928635 0.0814272655 0.558068632      2
R-MMU-1605595.Intestine     R-MMU-3928639 0.0351439091 0.871292666      7
R-MMU-1605595.Lung          R-MMU-3928641 0.0321227190 0.895690501     13
R-MMU-1605595.Muscle        R-MMU-3928

R-MMU-6798751.Mammary gland R-MMU-8948234 0.0014318609 0.939679873     17
R-MMU-6798751.Tendon         R-MMU-200652 0.0004769500 0.038938150      1
R-MMU-6798751.VNO           R-HSA-9831702 0.0000000000 0.037852837      1
R-MMU-9840795.Liver         R-MMU-1655831 0.1005186825 0.503656138      3
R-MMU-9840795.Brain         R-MMU-2065539 0.1415340959 0.632236387      4
R-MMU-9840795.Intestine     R-MMU-2065550 0.0669519478 0.503211094      3
R-MMU-9840795.Lung           R-MMU-211191 0.0171729758 0.244056065      3
R-MMU-9840795.Muscle         R-MMU-211206 0.0411883042 0.243663725      3
R-MMU-9840795.Testes        R-MMU-9029667 0.0069947669 0.243760223      3
R-MMU-9840795.Heart          R-MMU-909721 0.0303786932 0.540050993      3
R-MMU-9840795.Skin           R-MMU-913529 0.0278314369 0.037882899      1
R-MMU-9840795.Kidney        R-MMU-9624798 0.0295858067 0.037792268      1
R-MMU-9840795.Uterus        R-MMU-9710963 0.0087443219 0.540321075      3
R-MMU-9840795.Sperm         R-MMU-9734

...

R-MMU-8981926.Heart          R-MMU-399998 0.0410321016 0.518992242      4
R-MMU-8981926.Skin           R-MMU-445075 0.0257643958 0.395260036      2
R-MMU-8981926.Kidney        R-MMU-2468040 0.0807814397 0.503759483      4
R-MMU-8981926.Uterus        R-MMU-2470935 0.0090623014 0.618866540      5
R-MMU-8981926.Sperm         R-MMU-2545203 0.0135196489 0.503759483      4
R-MMU-8981926.Eye           R-MMU-2545253 0.0268734792 0.503759483      4
R-MMU-8981926.Adipose       R-MMU-1482695 0.1105264628 0.184563339      1
R-MMU-8981926.Adrenal         R-MMU-75886 0.0049277169 0.184646672      1
R-MMU-8981926.Ovary         R-MMU-4411383 0.0168585445 0.049686282      1
R-MMU-8981926.Olfactory     R-MMU-5334050 0.0057236437 0.439174426      2
R-MMU-8981926.Pancreas       R-MMU-212007 0.0410338762 0.347484366      2
R-MMU-8981926.Bone          R-MMU-5362525 0.0092238123 0.347929754      2
R-MMU-8981926.Nerve         R-MMU-2404134 0.0095417765 0.590312386      2
R-MMU-8981926.Stomach       R-MMU-2404

R-MMU-9013145.Adipose       R-NUL-5610751 0.0060401051 0.088348496      1
R-MMU-9013145.Adrenal       R-MMU-1169398 0.0004777094 0.334448086      3
R-MMU-9013145.Ovary         R-MMU-1169403 0.0049272210 0.143670943      2
R-MMU-9013145.Olfactory     R-MMU-1169405 0.0006354253 0.351347875      3
R-MMU-9013145.Pancreas      R-MMU-1678842 0.0036563607 0.643488525      6
R-MMU-9013145.Bone          R-MMU-1678843 0.0009539007 0.285777766      3
R-MMU-9013145.Nerve         R-MMU-5696021 0.0003182182 0.064663159      1
R-MMU-9013145.Stomach       R-MMU-1234163 0.0023844974 0.789205938      7
R-MMU-9013145.MOE           R-MMU-1234173 0.0000000000 0.728899187      5
R-MMU-9013145.Spinal cord   R-MMU-1234177 0.0015905854 0.729069968      5
R-MMU-9013145.Aorta         R-MMU-1234183 0.0006359325 0.729446732      5
R-MMU-9013145.Mammary gland R-MMU-1676114 0.0007954197 0.055562414      1
R-MMU-9013145.Tendon        R-MMU-1676124 0.0017493212 0.055748900      1
R-MMU-9013145.VNO           R-MMU-1676

R-MMU-202233.Adrenal        R-MMU-1358791 0.0038183835 0.798091421      4
R-MMU-202233.Ovary          R-MMU-1445150 0.0122417114 0.510131891      3
R-MMU-202233.Olfactory      R-MMU-1497784 0.0047710104 0.753235371      5
R-MMU-202233.Pancreas       R-MMU-1497796 0.0270342451 0.753589572      5
R-MMU-202233.Bone           R-MMU-1497810 0.0063623617 0.752612034      5
R-MMU-202233.Nerve           R-MMU-162352 0.0054084723 0.036948881      1
R-MMU-202233.Stomach         R-MMU-198599 0.0077924545 0.694775298      3
R-MMU-202233.MOE             R-MMU-198609 0.0090622966 0.777456554      4
R-MMU-202233.Spinal cord     R-MMU-198611 0.0120885483 0.785080147      4
R-MMU-202233.Aorta           R-MMU-198613 0.0079519450 0.770291783      4
R-MMU-202233.Mammary gland   R-MMU-198621 0.0027032259 0.693705110      3
R-MMU-202233.Tendon          R-MMU-198640 0.0054069469 0.851070808      5
R-MMU-202233.VNO             R-MMU-199298 0.0062005977 0.693715893      3
R-MMU-202245.Liver           R-MMU-199

R-MMU-202307.Stomach        R-MMU-2424476 0.0079494134 0.617328279      5
R-MMU-202307.MOE            R-MMU-2730840 0.0170147334 0.648708636      5
R-MMU-202307.Spinal cord    R-MMU-2730856 0.0171785404 0.804506989      7
R-MMU-202307.Aorta          R-MMU-2730889 0.0084276379 0.804724087      7
R-MMU-202307.Mammary gland   R-MMU-376123 0.0025432292 0.319530526      2
R-MMU-202307.Tendon         R-MMU-3858475 0.0063651433 0.687293007      4
R-MMU-202307.VNO             R-MMU-389348 0.0081101744 0.600828931      4
R-MMU-202344.Liver           R-MMU-389788 0.0356332647 0.663686654      4
R-MMU-202344.Brain          R-MMU-3928619 0.0777784923 0.319101699      2
R-MMU-202344.Intestine      R-MMU-3928620 0.0359401044 0.319372475      2
R-MMU-202344.Lung           R-MMU-3928642 0.0182863765 0.910022123      9
R-MMU-202344.Muscle          R-MMU-399930 0.0381648341 0.861115651      9
R-MMU-202344.Testes          R-MMU-399939 0.0162236351 0.861115651      9
R-MMU-202344.Heart           R-MMU-399

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[["R-MMU-450466"]], ]), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = GEO_model_training_tissue_detail.vec.train
)
ggplot(pca.d) +
   geom_point(aes(x = PC1, y = PC2, colour = Section)) +
   theme_bw()gai
p <- plot_ly(
   x = pca.d$PC1,
   y = pca.d$PC2,
   z = pca.d$PC3,
   type = "scatter3d",
   mode = "markers",
   color = pca.d$Section,
   size = 1
)
saveWidget(p, "plotly_chart_80.html", selfcontained = TRUE)

In addition: Warning messages:
1: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

2: In RColorBrewer::brewer.pal(N, "Set2") :
  n too large, allowed maximum for palette Set2 is 8
Returning the palette you asked for with that many colors

